In [1]:
import os
import cv2
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

# ==========================
# CONFIG
# ==========================
DATASET_DIR = r"C:\Preet\9000_paired_bscans"  # both *_l.png and *_h.png are here
IMAGE_SIZE = (256, 256)
BATCH_SIZE = 16
EPOCHS = 75
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Running on:", DEVICE)

MODEL_PATH = os.path.join(DATASET_DIR, "best_attention_unet_final_huber+ssim.pth")
RESULTS_DIR = os.path.join(DATASET_DIR, "predictions_attention_final_huber_ssim")
os.makedirs(RESULTS_DIR, exist_ok=True)

# ==========================
# DATASET
# ==========================
class GPRDataset(Dataset):
    def __init__(self, x_paths, y_paths):
        self.x_paths = x_paths
        self.y_paths = y_paths

    def __len__(self):
        return len(self.x_paths)

    def __getitem__(self, idx):
        x = np.array(Image.open(self.x_paths[idx]).resize(IMAGE_SIZE), dtype=np.float32) / 255.0
        y = np.array(Image.open(self.y_paths[idx]).resize(IMAGE_SIZE), dtype=np.float32) / 255.0
        x = torch.tensor(x).unsqueeze(0)  # (1,H,W)
        y = torch.tensor(y).unsqueeze(0)
        return x, y

def load_data(dataset_dir):
    low_paths, high_paths = [], []
    for file in os.listdir(dataset_dir):
        if file.endswith("_l.png"):
            low_path = os.path.join(dataset_dir, file)
            high_path = os.path.join(dataset_dir, file.replace("_l.png", "_h.png"))
            if os.path.exists(high_path):
                low_paths.append(low_path)
                high_paths.append(high_path)
    return low_paths, high_paths

all_x, all_y = load_data(DATASET_DIR)

# Split 70/15/15
train_x, temp_x, train_y, temp_y = train_test_split(all_x, all_y, test_size=0.30, random_state=42)
val_x, test_x, val_y, test_y = train_test_split(temp_x, temp_y, test_size=0.50, random_state=42)

print(f"Train: {len(train_x)}, Val: {len(val_x)}, Test: {len(test_x)}")

train_loader = DataLoader(GPRDataset(train_x, train_y), batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(GPRDataset(val_x, val_y), batch_size=BATCH_SIZE, shuffle=False)
test_loader  = DataLoader(GPRDataset(test_x, test_y), batch_size=1, shuffle=False)

# ==========================
# MODEL BLOCKS
# ==========================
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, padding=1),
            nn.ReLU(inplace=True),
        )
    def forward(self, x):
        return self.conv(x)

class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(channels, channels, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),
            nn.Conv2d(channels, channels, 3, padding=1),
        )
    def forward(self, x):
        return x + self.conv(x)

class AttentionGate(nn.Module):
    def __init__(self, F_g, F_l, F_int):
        super().__init__()
        self.W_g = nn.Conv2d(F_g, F_int, kernel_size=1)
        self.W_x = nn.Conv2d(F_l, F_int, kernel_size=1)
        self.psi = nn.Sequential(
            nn.ReLU(inplace=True),
            nn.Conv2d(F_int, 1, kernel_size=1),
            nn.Sigmoid()
        )
    def forward(self, x, g):
        g1 = self.W_g(g)
        x1 = self.W_x(x)
        psi = self.psi(g1 + x1)
        return x * psi

# ==========================
# UNet-3 with Attention + Residual Bottleneck
# ==========================
class UNet3WithAttention(nn.Module):
    def __init__(self):
        super().__init__()
        # Encoder (3 levels)
        self.enc1 = ConvBlock(1, 64); self.pool1 = nn.MaxPool2d(2)
        self.enc2 = ConvBlock(64, 128); self.pool2 = nn.MaxPool2d(2)
        self.enc3 = ConvBlock(128, 256); self.pool3 = nn.MaxPool2d(2)

        # Bottleneck with residual blocks (FIX: 256 channels)
        self.bottleneck = nn.Sequential(*[ResidualBlock(256) for _ in range(6)])

        # Decoder
        self.up3 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.att3 = AttentionGate(256, 256, 128)   # bottleneck=256, skip=256
        self.dec3 = ConvBlock(256+256, 256)        # concat → 512

        self.up2 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.att2 = AttentionGate(256, 128, 64)
        self.dec2 = ConvBlock(256+128, 128)

        self.up1 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.att1 = AttentionGate(128, 64, 32)
        self.dec1 = ConvBlock(128+64, 64)

        self.out_conv = nn.Conv2d(64, 1, kernel_size=1)

    def forward(self, x):
        # Encoder
        c1 = self.enc1(x); p1 = self.pool1(c1)
        c2 = self.enc2(p1); p2 = self.pool2(c2)
        c3 = self.enc3(p2); p3 = self.pool3(c3)

        # Bottleneck
        b = self.bottleneck(p3)

        # Decoder
        u3 = self.up3(b); a3 = self.att3(c3, u3); d3 = self.dec3(torch.cat([u3, a3], dim=1))
        u2 = self.up2(d3); a2 = self.att2(c2, u2); d2 = self.dec2(torch.cat([u2, a2], dim=1))
        u1 = self.up1(d2); a1 = self.att1(c1, u1); d1 = self.dec1(torch.cat([u1, a1], dim=1))

        return self.out_conv(d1)


# ==========================
# TRAINING
# ==========================
model = UNet3WithAttention().to(DEVICE)
from pytorch_msssim import ssim  # pip install pytorch-msssim

alpha = 0.8
beta  = 0.2

criterion = nn.HuberLoss(delta=1.0)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

best_val_loss = float("inf")

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0.0
    for x, y in train_loader:
        x, y = x.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad()
        preds = model(x)

        # Huber Loss
        huber_loss = criterion(preds, y)

        # SSIM Loss (1 - SSIM)
        ssim_loss = 1 - ssim(preds, y, data_range=1.0, size_average=True)  # assumes preds/y in [0,1]

        # Combined Loss
        loss = alpha * huber_loss + beta * ssim_loss
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * x.size(0)
    train_loss /= len(train_loader.dataset)

    # ---- Validation ----
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(DEVICE), y.to(DEVICE)
            preds = model(x)

            huber_loss = criterion(preds, y)
            ssim_loss = 1 - ssim(preds, y, data_range=1.0, size_average=True)
            loss = alpha * huber_loss + beta * ssim_loss

            val_loss += loss.item() * x.size(0)
    val_loss /= len(val_loader.dataset)

    print(f"Epoch [{epoch+1}/{EPOCHS}] Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), MODEL_PATH)
        print(f"  ✅ Saved Best Model at Epoch {epoch+1}")

# ==========================
# INFERENCE
# ==========================
print("\nRunning inference on test set...")
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.eval()

for i, (x, y) in enumerate(test_loader):
    x = x.to(DEVICE)
    with torch.no_grad():
        pred = model(x).cpu().squeeze(0).squeeze(0).numpy()
    pred_img = (pred * 255.0).clip(0, 255).astype(np.uint8)
    Image.fromarray(pred_img).save(os.path.join(RESULTS_DIR, f"pred_{i+1}.png"))

print(f"Predictions saved in {RESULTS_DIR}")

Running on: cuda
Train: 6300, Val: 1350, Test: 1350


KeyboardInterrupt: 

In [ ]:
import os
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
from math import log10


import shutil
os.makedirs(r"C:\Preet\9000_paired_bscans\ground_truth_test_final_huber+ssim", exist_ok=True)

for f in test_y:  # list of test high-res paths from your train/val/test split
    shutil.copy(f, r"C:\Preet\9000_paired_bscans\ground_truth_test_final_huber+ssim")


# ==== PATHS ====
pred_dir = r"C:\Preet\9000_paired_bscans\predictions_attention_final_huber_ssim"
gt_dir   = r"C:\Preet\9000_paired_bscans\ground_truth_test_final_huber+ssim"

# ==== FUNCTIONS ====
def calculate_psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0:
        return float('inf')
    return 20 * log10(255.0 / np.sqrt(mse))

def calculate_ssim(img1, img2):
    return ssim(img1, img2, data_range=255)

# ==== MAIN ====
psnr_values, ssim_values = [], []

pred_files = sorted([f for f in os.listdir(pred_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
gt_files   = sorted([f for f in os.listdir(gt_dir)   if f.lower().endswith(('.png', '.jpg', '.jpeg'))])

num_pairs = min(len(pred_files), len(gt_files))
if num_pairs == 0:
    print("[Error] No matching image files found in both directories.")
else:
    if len(pred_files) != len(gt_files):
        print(f"[Warning] Different number of images: Predictions={len(pred_files)}, Ground Truth={len(gt_files)}")
        print(f"Evaluating only first {num_pairs} matched pairs.")

    for i in range(num_pairs):
        pred_path = os.path.join(pred_dir, pred_files[i])
        gt_path   = os.path.join(gt_dir, gt_files[i])

        pred_img = cv2.imread(pred_path, cv2.IMREAD_GRAYSCALE)
        gt_img   = cv2.imread(gt_path, cv2.IMREAD_GRAYSCALE)

        if pred_img is None or gt_img is None:
            print(f"[Error] Could not load: {pred_files[i]} or {gt_files[i]}")
            continue

        if pred_img.shape != gt_img.shape:
            pred_img = cv2.resize(pred_img, (gt_img.shape[1], gt_img.shape[0]))

        psnr_values.append(calculate_psnr(pred_img, gt_img))
        ssim_values.append(calculate_ssim(pred_img, gt_img))

    if psnr_values and ssim_values:
        print(f"\n---- Test Set Evaluation ----")
        print(f"SSIM: avg={np.mean(ssim_values):.4f}, min={np.min(ssim_values):.4f}, max={np.max(ssim_values):.4f}")
        print(f"PSNR: avg={np.mean(psnr_values):.2f} dB, min={np.min(psnr_values):.2f} dB, max={np.max(psnr_values):.2f} dB")


---- Test Set Evaluation ----
SSIM: avg=0.8937, min=0.7131, max=0.9990
PSNR: avg=35.48 dB, min=28.60 dB, max=56.07 dB


In [ ]:
import os
import re
import numpy as np
from PIL import Image
import cv2
from math import log10
from skimage.metrics import structural_similarity as ssim

import torch
import torch.nn as nn

# =========================
# CONFIG
# =========================
FREQ_DIRS = [
    r"C:\Preet\validation dataset\png_images_650M_1083M",
    r"C:\Preet\validation dataset\png_images_700M_1167M",
    r"C:\Preet\validation dataset\png_images_800M_1333M",
    r"C:\Preet\validation dataset\png_images_850M_1416M",
    r"C:\Preet\validation dataset\png_images_900M_1500M",
]

MODEL_PATH = r"C:\Preet\9000_paired_bscans\best_attention_unet_final_huber+ssim.pth"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
IMAGE_SIZE = (256, 256)

# =========================
# MODEL BLOCKS (UNet3 + 6 RB)
# =========================
class ConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.ReLU(inplace=True),
        )
    def forward(self, x): return self.conv(x)

class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(channels, channels, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),
            nn.Conv2d(channels, channels, 3, padding=1),
        )
    def forward(self, x): return x + self.conv(x)

class AttentionGate(nn.Module):
    def __init__(self, F_g, F_l, F_int):
        super().__init__()
        self.W_g = nn.Conv2d(F_g, F_int, 1)
        self.W_x = nn.Conv2d(F_l, F_int, 1)
        self.psi = nn.Sequential(
            nn.ReLU(inplace=True),
            nn.Conv2d(F_int, 1, 1),
            nn.Sigmoid()
        )
    def forward(self, x, g):
        g1 = self.W_g(g)
        x1 = self.W_x(x)
        psi = self.psi(g1 + x1)
        return x * psi

class UNet3WithAttention(nn.Module):
    def __init__(self):
        super().__init__()
        # Encoder
        self.enc1 = ConvBlock(1, 64); self.pool1 = nn.MaxPool2d(2)
        self.enc2 = ConvBlock(64, 128); self.pool2 = nn.MaxPool2d(2)
        self.enc3 = ConvBlock(128, 256); self.pool3 = nn.MaxPool2d(2)

        # Bottleneck
        self.bottleneck = nn.Sequential(*[ResidualBlock(256) for _ in range(6)])

        # Decoder
        self.up3 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.att3 = AttentionGate(256, 256, 128)
        self.dec3 = ConvBlock(512, 256)

        self.up2 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.att2 = AttentionGate(256, 128, 64)
        self.dec2 = ConvBlock(384, 128)

        self.up1 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.att1 = AttentionGate(128, 64, 32)
        self.dec1 = ConvBlock(192, 64)

        self.out_conv = nn.Conv2d(64, 1, 1)

    def forward(self, x):
        c1 = self.enc1(x); p1 = self.pool1(c1)
        c2 = self.enc2(p1); p2 = self.pool2(c2)
        c3 = self.enc3(p2); p3 = self.pool3(c3)
        b = self.bottleneck(p3)

        u3 = self.up3(b); a3 = self.att3(c3, u3); d3 = self.dec3(torch.cat([u3, a3], dim=1))
        u2 = self.up2(d3); a2 = self.att2(c2, u2); d2 = self.dec2(torch.cat([u2, a2], dim=1))
        u1 = self.up1(d2); a1 = self.att1(c1, u1); d1 = self.dec1(torch.cat([u1, a1], dim=1))
        return self.out_conv(d1)

# =========================
# HELPERS
# =========================
def calculate_psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0: return float('inf')
    return 20 * log10(255.0 / np.sqrt(mse))

def calculate_ssim(img1, img2):
    return ssim(img1, img2, data_range=255)

def numerical_sort(files):
    return sorted(files, key=lambda f: int(re.search(r'\d+', f).group()))

# =========================
# LOAD MODEL
# =========================
model = UNet3WithAttention().to(DEVICE)
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.eval()

# =========================
# INFERENCE LOOP
# =========================
for freq_dir in FREQ_DIRS:
    print(f"\n---- Evaluating {os.path.basename(freq_dir)} ----")
    pred_dir = os.path.join(freq_dir, "predictions_attention_unet_final_huber+ssim")
    os.makedirs(pred_dir, exist_ok=True)

    low_files = numerical_sort([f for f in os.listdir(freq_dir) if f.endswith("_l.png")])
    high_files = numerical_sort([f for f in os.listdir(freq_dir) if f.endswith("_h.png")])

    psnr_values, ssim_values = [], []

    for i in range(len(low_files)):
        low_path = os.path.join(freq_dir, low_files[i])
        high_path = os.path.join(freq_dir, high_files[i])

        # Load LR
        lr_img = np.array(Image.open(low_path).resize(IMAGE_SIZE), dtype=np.float32) / 255.0
        lr_tensor = torch.tensor(lr_img).unsqueeze(0).unsqueeze(0).to(DEVICE)

        # Predict
        with torch.no_grad():
            pred = model(lr_tensor).cpu().numpy()

        # remove batch and channel dims, scale to 0-255
        pred_img = np.squeeze(pred)
        pred_img = (pred_img * 255.0).clip(0, 255).astype(np.uint8)

        # increase contrast (so it's dark like GT)
        pred_img = cv2.normalize(pred_img, None, 0, 255, cv2.NORM_MINMAX)

        # save prediction
        pred_path = os.path.join(pred_dir, f"pred_{i+1}.png")
        Image.fromarray(pred_img).save(pred_path)

        # Load GT
        gt_img = cv2.imread(high_path, cv2.IMREAD_GRAYSCALE)
        if pred_img.shape != gt_img.shape:
            pred_img = cv2.resize(pred_img, (gt_img.shape[1], gt_img.shape[0]))

        # Metrics
        psnr_values.append(calculate_psnr(pred_img, gt_img))
        ssim_values.append(calculate_ssim(pred_img, gt_img))

    # Results
    if psnr_values and ssim_values:
        print(f"SSIM: avg={np.mean(ssim_values):.4f}, min={np.min(ssim_values):.4f}, max={np.max(ssim_values):.4f}")
        print(f"PSNR: avg={np.mean(psnr_values):.2f} dB, min={np.min(psnr_values):.2f} dB, max={np.max(psnr_values):.2f} dB")
    else:
        print("[Error] No valid pairs processed.")



---- Evaluating png_images_650M_1083M ----
SSIM: avg=0.9544, min=0.9236, max=0.9669
PSNR: avg=35.66 dB, min=31.27 dB, max=37.21 dB

---- Evaluating png_images_700M_1167M ----
SSIM: avg=0.9878, min=0.7614, max=0.9926
PSNR: avg=38.11 dB, min=29.32 dB, max=39.10 dB

---- Evaluating png_images_800M_1333M ----
SSIM: avg=0.9903, min=0.9791, max=0.9934
PSNR: avg=40.32 dB, min=35.01 dB, max=41.66 dB

---- Evaluating png_images_850M_1416M ----
SSIM: avg=0.9650, min=0.9424, max=0.9752
PSNR: avg=34.47 dB, min=32.15 dB, max=35.33 dB

---- Evaluating png_images_900M_1500M ----
SSIM: avg=0.9314, min=0.8822, max=0.9506
PSNR: avg=32.69 dB, min=29.83 dB, max=33.63 dB


In [ ]:
import os
import re
import numpy as np
from PIL import Image
import cv2
from math import log10
from skimage.metrics import structural_similarity as ssim

import torch
import torch.nn as nn

# =========================
# CONFIG
# =========================
DATASET_DIR = r"C:\Preet\validation dataset\png_images_650M_1083M_1800M"
MODEL_PATH = r"C:\Preet\9000_paired_bscans\best_attention_unet_final_huber+ssim.pth"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
IMAGE_SIZE = (256, 256)

# =========================
# MODEL BLOCKS
# =========================
class ConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.ReLU(inplace=True),
        )
    def forward(self, x): return self.conv(x)

class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(channels, channels, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),
            nn.Conv2d(channels, channels, 3, padding=1),
        )
    def forward(self, x): return x + self.conv(x)

class AttentionGate(nn.Module):
    def __init__(self, F_g, F_l, F_int):
        super().__init__()
        self.W_g = nn.Conv2d(F_g, F_int, 1)
        self.W_x = nn.Conv2d(F_l, F_int, 1)
        self.psi = nn.Sequential(
            nn.ReLU(inplace=True),
            nn.Conv2d(F_int, 1, 1),
            nn.Sigmoid()
        )
    def forward(self, x, g):
        g1 = self.W_g(g)
        x1 = self.W_x(x)
        psi = self.psi(g1 + x1)
        return x * psi

class UNet3WithAttention(nn.Module):
    def __init__(self):
        super().__init__()
        # Encoder
        self.enc1 = ConvBlock(1, 64); self.pool1 = nn.MaxPool2d(2)
        self.enc2 = ConvBlock(64, 128); self.pool2 = nn.MaxPool2d(2)
        self.enc3 = ConvBlock(128, 256); self.pool3 = nn.MaxPool2d(2)

        # Bottleneck
        self.bottleneck = nn.Sequential(*[ResidualBlock(256) for _ in range(6)])

        # Decoder
        self.up3 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.att3 = AttentionGate(256, 256, 128)
        self.dec3 = ConvBlock(512, 256)

        self.up2 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.att2 = AttentionGate(256, 128, 64)
        self.dec2 = ConvBlock(384, 128)

        self.up1 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.att1 = AttentionGate(128, 64, 32)
        self.dec1 = ConvBlock(192, 64)

        self.out_conv = nn.Conv2d(64, 1, 1)

    def forward(self, x):
        c1 = self.enc1(x); p1 = self.pool1(c1)
        c2 = self.enc2(p1); p2 = self.pool2(c2)
        c3 = self.enc3(p2); p3 = self.pool3(c3)
        b = self.bottleneck(p3)

        u3 = self.up3(b); a3 = self.att3(c3, u3); d3 = self.dec3(torch.cat([u3, a3], dim=1))
        u2 = self.up2(d3); a2 = self.att2(c2, u2); d2 = self.dec2(torch.cat([u2, a2], dim=1))
        u1 = self.up1(d2); a1 = self.att1(c1, u1); d1 = self.dec1(torch.cat([u1, a1], dim=1))
        return self.out_conv(d1)

# =========================
# HELPERS
# =========================
def calculate_psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0: return float('inf')
    return 20 * log10(255.0 / np.sqrt(mse))

def calculate_ssim(img1, img2):
    return ssim(img1, img2, data_range=255)

def numerical_sort(files):
    return sorted(files, key=lambda f: int(re.search(r'\d+', f).group()))

def run_model(img, model):
    tensor = torch.tensor(img, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(DEVICE)
    with torch.no_grad():
        pred = model(tensor).cpu().numpy()
    pred = np.squeeze(pred)
    pred = (pred * 255.0).clip(0, 255).astype(np.uint8)
    return cv2.normalize(pred, None, 0, 255, cv2.NORM_MINMAX)

# =========================
# LOAD MODEL
# =========================
model = UNet3WithAttention().to(DEVICE)
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.eval()

# =========================
# INFERENCE 2-STAGE
# =========================
files_650 = numerical_sort([f for f in os.listdir(DATASET_DIR) if f.endswith("650_bscan.png")])
files_1083 = numerical_sort([f for f in os.listdir(DATASET_DIR) if f.endswith("1083_bscan.png")])
files_1800 = numerical_sort([f for f in os.listdir(DATASET_DIR) if f.endswith("1800_bscan.png")])

psnr_stage1, ssim_stage1 = [], []
psnr_stage2, ssim_stage2 = [], []

pred_dir1 = os.path.join(DATASET_DIR, "predictions_stage1_huber+loss")
pred_dir2 = os.path.join(DATASET_DIR, "predictions_stage2_huber+loss")
os.makedirs(pred_dir1, exist_ok=True)
os.makedirs(pred_dir2, exist_ok=True)

for i in range(len(files_650)):
    # ---- Stage 1: 650 -> pred -> compare with 1083 ----
    lr_img = np.array(Image.open(os.path.join(DATASET_DIR, files_650[i])).resize(IMAGE_SIZE), dtype=np.float32) / 255.0
    gt_1083 = cv2.imread(os.path.join(DATASET_DIR, files_1083[i]), cv2.IMREAD_GRAYSCALE)

    pred_1083 = run_model(lr_img, model)
    Image.fromarray(pred_1083).save(os.path.join(pred_dir1, f"pred1_{i+1}.png"))

    if pred_1083.shape != gt_1083.shape:
        pred_1083 = cv2.resize(pred_1083, (gt_1083.shape[1], gt_1083.shape[0]))

    psnr_stage1.append(calculate_psnr(pred_1083, gt_1083))
    ssim_stage1.append(calculate_ssim(pred_1083, gt_1083))

    # ---- Stage 2: pred_1083 -> pred -> compare with 1800 ----
    gt_1800 = cv2.imread(os.path.join(DATASET_DIR, files_1800[i]), cv2.IMREAD_GRAYSCALE)

    # resize pred_1083 to IMAGE_SIZE before refeeding
    pred_1083_resized = cv2.resize(pred_1083, IMAGE_SIZE).astype(np.float32) / 255.0
    pred_1800 = run_model(pred_1083_resized, model)
    Image.fromarray(pred_1800).save(os.path.join(pred_dir2, f"pred2_{i+1}.png"))

    if pred_1800.shape != gt_1800.shape:
        pred_1800 = cv2.resize(pred_1800, (gt_1800.shape[1], gt_1800.shape[0]))

    psnr_stage2.append(calculate_psnr(pred_1800, gt_1800))
    ssim_stage2.append(calculate_ssim(pred_1800, gt_1800))

# =========================
# RESULTS
# =========================
print("\n---- Stage 1: 650 → pred → compare with 1083 ----")
print(f"SSIM: avg={np.mean(ssim_stage1):.4f}, min={np.min(ssim_stage1):.4f}, max={np.max(ssim_stage1):.4f}")
print(f"PSNR: avg={np.mean(psnr_stage1):.2f} dB, min={np.min(psnr_stage1):.2f} dB, max={np.max(psnr_stage1):.2f} dB")

print("\n---- Stage 2: pred_1083 → pred → compare with 1800 ----")
print(f"SSIM: avg={np.mean(ssim_stage2):.4f}, min={np.min(ssim_stage2):.4f}, max={np.max(ssim_stage2):.4f}")
print(f"PSNR: avg={np.mean(psnr_stage2):.2f} dB, min={np.min(psnr_stage2):.2f} dB, max={np.max(psnr_stage2):.2f} dB")



---- Stage 1: 650 → pred → compare with 1083 ----
SSIM: avg=0.9523, min=0.9242, max=0.9662
PSNR: avg=35.50 dB, min=31.40 dB, max=36.76 dB

---- Stage 2: pred_1083 → pred → compare with 1800 ----
SSIM: avg=0.9346, min=0.9011, max=0.9548
PSNR: avg=29.87 dB, min=29.29 dB, max=31.15 dB


In [ ]:
import os
import re
import numpy as np
from PIL import Image
import cv2
from math import log10
from skimage.metrics import structural_similarity as ssim

import torch
import torch.nn as nn

# =========================
# CONFIG
# =========================
DATASET_DIR = r"C:\Preet\validation dataset\png_images_900M_1500M_2500M"
MODEL_PATH = r"C:\Preet\9000_paired_bscans\best_attention_unet_final_huber+ssim.pth"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
IMAGE_SIZE = (256, 256)

# =========================
# MODEL BLOCKS
# =========================
class ConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.ReLU(inplace=True),
        )
    def forward(self, x): return self.conv(x)

class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(channels, channels, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),
            nn.Conv2d(channels, channels, 3, padding=1),
        )
    def forward(self, x): return x + self.conv(x)

class AttentionGate(nn.Module):
    def __init__(self, F_g, F_l, F_int):
        super().__init__()
        self.W_g = nn.Conv2d(F_g, F_int, 1)
        self.W_x = nn.Conv2d(F_l, F_int, 1)
        self.psi = nn.Sequential(
            nn.ReLU(inplace=True),
            nn.Conv2d(F_int, 1, 1),
            nn.Sigmoid()
        )
    def forward(self, x, g):
        g1 = self.W_g(g)
        x1 = self.W_x(x)
        psi = self.psi(g1 + x1)
        return x * psi

class UNet3WithAttention(nn.Module):
    def __init__(self):
        super().__init__()
        # Encoder
        self.enc1 = ConvBlock(1, 64); self.pool1 = nn.MaxPool2d(2)
        self.enc2 = ConvBlock(64, 128); self.pool2 = nn.MaxPool2d(2)
        self.enc3 = ConvBlock(128, 256); self.pool3 = nn.MaxPool2d(2)

        # Bottleneck
        self.bottleneck = nn.Sequential(*[ResidualBlock(256) for _ in range(6)])

        # Decoder
        self.up3 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.att3 = AttentionGate(256, 256, 128)
        self.dec3 = ConvBlock(512, 256)

        self.up2 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.att2 = AttentionGate(256, 128, 64)
        self.dec2 = ConvBlock(384, 128)

        self.up1 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.att1 = AttentionGate(128, 64, 32)
        self.dec1 = ConvBlock(192, 64)

        self.out_conv = nn.Conv2d(64, 1, 1)

    def forward(self, x):
        c1 = self.enc1(x); p1 = self.pool1(c1)
        c2 = self.enc2(p1); p2 = self.pool2(c2)
        c3 = self.enc3(p2); p3 = self.pool3(c3)
        b = self.bottleneck(p3)

        u3 = self.up3(b); a3 = self.att3(c3, u3); d3 = self.dec3(torch.cat([u3, a3], dim=1))
        u2 = self.up2(d3); a2 = self.att2(c2, u2); d2 = self.dec2(torch.cat([u2, a2], dim=1))
        u1 = self.up1(d2); a1 = self.att1(c1, u1); d1 = self.dec1(torch.cat([u1, a1], dim=1))
        return self.out_conv(d1)

# =========================
# HELPERS
# =========================
def calculate_psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0: return float('inf')
    return 20 * log10(255.0 / np.sqrt(mse))

def calculate_ssim(img1, img2):
    return ssim(img1, img2, data_range=255)

def numerical_sort(files):
    return sorted(files, key=lambda f: int(re.search(r'\d+', f).group()))

def run_model(img, model):
    tensor = torch.tensor(img, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(DEVICE)
    with torch.no_grad():
        pred = model(tensor).cpu().numpy()
    pred = np.squeeze(pred)
    pred = (pred * 255.0).clip(0, 255).astype(np.uint8)
    return cv2.normalize(pred, None, 0, 255, cv2.NORM_MINMAX)

# =========================
# LOAD MODEL
# =========================
model = UNet3WithAttention().to(DEVICE)
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.eval()

# =========================
# INFERENCE 2-STAGE
# =========================
files_650 = numerical_sort([f for f in os.listdir(DATASET_DIR) if f.endswith("900_bscan.png")])
files_1083 = numerical_sort([f for f in os.listdir(DATASET_DIR) if f.endswith("1500_bscan.png")])
files_1800 = numerical_sort([f for f in os.listdir(DATASET_DIR) if f.endswith("2500_bscan.png")])

psnr_stage1, ssim_stage1 = [], []
psnr_stage2, ssim_stage2 = [], []

pred_dir1 = os.path.join(DATASET_DIR, "predictions_stage1_huber+loss")
pred_dir2 = os.path.join(DATASET_DIR, "predictions_stage2_huber+loss")
os.makedirs(pred_dir1, exist_ok=True)
os.makedirs(pred_dir2, exist_ok=True)

for i in range(len(files_650)):
    # ---- Stage 1: 650 -> pred -> compare with 1083 ----
    lr_img = np.array(Image.open(os.path.join(DATASET_DIR, files_650[i])).resize(IMAGE_SIZE), dtype=np.float32) / 255.0
    gt_1083 = cv2.imread(os.path.join(DATASET_DIR, files_1083[i]), cv2.IMREAD_GRAYSCALE)

    pred_1083 = run_model(lr_img, model)
    Image.fromarray(pred_1083).save(os.path.join(pred_dir1, f"pred1_{i+1}.png"))

    if pred_1083.shape != gt_1083.shape:
        pred_1083 = cv2.resize(pred_1083, (gt_1083.shape[1], gt_1083.shape[0]))

    psnr_stage1.append(calculate_psnr(pred_1083, gt_1083))
    ssim_stage1.append(calculate_ssim(pred_1083, gt_1083))

    # ---- Stage 2: pred_1083 -> pred -> compare with 1800 ----
    gt_1800 = cv2.imread(os.path.join(DATASET_DIR, files_1800[i]), cv2.IMREAD_GRAYSCALE)

    # resize pred_1083 to IMAGE_SIZE before refeeding
    pred_1083_resized = cv2.resize(pred_1083, IMAGE_SIZE).astype(np.float32) / 255.0
    pred_1800 = run_model(pred_1083_resized, model)
    Image.fromarray(pred_1800).save(os.path.join(pred_dir2, f"pred2_{i+1}.png"))

    if pred_1800.shape != gt_1800.shape:
        pred_1800 = cv2.resize(pred_1800, (gt_1800.shape[1], gt_1800.shape[0]))

    psnr_stage2.append(calculate_psnr(pred_1800, gt_1800))
    ssim_stage2.append(calculate_ssim(pred_1800, gt_1800))

# =========================
# RESULTS
# =========================
print("\n---- Stage 1: 900 → pred → compare with 1500 ----")
print(f"SSIM: avg={np.mean(ssim_stage1):.4f}, min={np.min(ssim_stage1):.4f}, max={np.max(ssim_stage1):.4f}")
print(f"PSNR: avg={np.mean(psnr_stage1):.2f} dB, min={np.min(psnr_stage1):.2f} dB, max={np.max(psnr_stage1):.2f} dB")

print("\n---- Stage 2: pred_1500 → pred → compare with 2500 ----")
print(f"SSIM: avg={np.mean(ssim_stage2):.4f}, min={np.min(ssim_stage2):.4f}, max={np.max(ssim_stage2):.4f}")
print(f"PSNR: avg={np.mean(psnr_stage2):.2f} dB, min={np.min(psnr_stage2):.2f} dB, max={np.max(psnr_stage2):.2f} dB")



---- Stage 1: 900 → pred → compare with 1500 ----
SSIM: avg=0.7241, min=0.5826, max=0.8610
PSNR: avg=29.10 dB, min=28.40 dB, max=29.87 dB

---- Stage 2: pred_1500 → pred → compare with 2500 ----
SSIM: avg=0.6782, min=0.5526, max=0.8319
PSNR: avg=27.56 dB, min=27.37 dB, max=27.94 dB


---------------------------

---------------------------

-------------------------

------------------------------------

In [1]:
import os
import cv2
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

# ==========================
# CONFIG
# ==========================
DATASET_DIR = r"C:\Preet\4000_paired_bscans"  # both *_l.png and *_h.png are here
IMAGE_SIZE = (256, 256)
BATCH_SIZE = 16
EPOCHS = 75
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Running on:", DEVICE)

MODEL_PATH = os.path.join(DATASET_DIR, "best_attention_unet_final_huber+ssim.pth")
RESULTS_DIR = os.path.join(DATASET_DIR, "predictions_attention_final_huber_ssim")
os.makedirs(RESULTS_DIR, exist_ok=True)

# ==========================
# DATASET
# ==========================
class GPRDataset(Dataset):
    def __init__(self, x_paths, y_paths):
        self.x_paths = x_paths
        self.y_paths = y_paths

    def __len__(self):
        return len(self.x_paths)

    def __getitem__(self, idx):
        x = np.array(Image.open(self.x_paths[idx]).resize(IMAGE_SIZE), dtype=np.float32) / 255.0
        y = np.array(Image.open(self.y_paths[idx]).resize(IMAGE_SIZE), dtype=np.float32) / 255.0
        x = torch.tensor(x).unsqueeze(0)  # (1,H,W)
        y = torch.tensor(y).unsqueeze(0)
        return x, y

def load_data(dataset_dir):
    low_paths, high_paths = [], []
    for file in os.listdir(dataset_dir):
        if file.endswith("_l.png"):
            low_path = os.path.join(dataset_dir, file)
            high_path = os.path.join(dataset_dir, file.replace("_l.png", "_h.png"))
            if os.path.exists(high_path):
                low_paths.append(low_path)
                high_paths.append(high_path)
    return low_paths, high_paths

all_x, all_y = load_data(DATASET_DIR)

# Split 70/15/15
train_x, temp_x, train_y, temp_y = train_test_split(all_x, all_y, test_size=0.30, random_state=42)
val_x, test_x, val_y, test_y = train_test_split(temp_x, temp_y, test_size=0.50, random_state=42)

print(f"Train: {len(train_x)}, Val: {len(val_x)}, Test: {len(test_x)}")

train_loader = DataLoader(GPRDataset(train_x, train_y), batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(GPRDataset(val_x, val_y), batch_size=BATCH_SIZE, shuffle=False)
test_loader  = DataLoader(GPRDataset(test_x, test_y), batch_size=1, shuffle=False)

# ==========================
# MODEL BLOCKS
# ==========================
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, padding=1),
            nn.ReLU(inplace=True),
        )
    def forward(self, x):
        return self.conv(x)

class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(channels, channels, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),
            nn.Conv2d(channels, channels, 3, padding=1),
        )
    def forward(self, x):
        return x + self.conv(x)

class AttentionGate(nn.Module):
    def __init__(self, F_g, F_l, F_int):
        super().__init__()
        self.W_g = nn.Conv2d(F_g, F_int, kernel_size=1)
        self.W_x = nn.Conv2d(F_l, F_int, kernel_size=1)
        self.psi = nn.Sequential(
            nn.ReLU(inplace=True),
            nn.Conv2d(F_int, 1, kernel_size=1),
            nn.Sigmoid()
        )
    def forward(self, x, g):
        g1 = self.W_g(g)
        x1 = self.W_x(x)
        psi = self.psi(g1 + x1)
        return x * psi

# ==========================
# UNet-3 with Attention + Residual Bottleneck
# ==========================
class UNet3WithAttention(nn.Module):
    def __init__(self):
        super().__init__()
        # Encoder (3 levels)
        self.enc1 = ConvBlock(1, 64); self.pool1 = nn.MaxPool2d(2)
        self.enc2 = ConvBlock(64, 128); self.pool2 = nn.MaxPool2d(2)
        self.enc3 = ConvBlock(128, 256); self.pool3 = nn.MaxPool2d(2)

        # Bottleneck with residual blocks (FIX: 256 channels)
        self.bottleneck = nn.Sequential(*[ResidualBlock(256) for _ in range(6)])

        # Decoder
        self.up3 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.att3 = AttentionGate(256, 256, 128)   # bottleneck=256, skip=256
        self.dec3 = ConvBlock(256+256, 256)        # concat → 512

        self.up2 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.att2 = AttentionGate(256, 128, 64)
        self.dec2 = ConvBlock(256+128, 128)

        self.up1 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.att1 = AttentionGate(128, 64, 32)
        self.dec1 = ConvBlock(128+64, 64)

        self.out_conv = nn.Conv2d(64, 1, kernel_size=1)

    def forward(self, x):
        # Encoder
        c1 = self.enc1(x); p1 = self.pool1(c1)
        c2 = self.enc2(p1); p2 = self.pool2(c2)
        c3 = self.enc3(p2); p3 = self.pool3(c3)

        # Bottleneck
        b = self.bottleneck(p3)

        # Decoder
        u3 = self.up3(b); a3 = self.att3(c3, u3); d3 = self.dec3(torch.cat([u3, a3], dim=1))
        u2 = self.up2(d3); a2 = self.att2(c2, u2); d2 = self.dec2(torch.cat([u2, a2], dim=1))
        u1 = self.up1(d2); a1 = self.att1(c1, u1); d1 = self.dec1(torch.cat([u1, a1], dim=1))

        return self.out_conv(d1)


# ==========================
# TRAINING
# ==========================
model = UNet3WithAttention().to(DEVICE)
from pytorch_msssim import ssim  # pip install pytorch-msssim

alpha = 0.8
beta  = 0.2

criterion = nn.HuberLoss(delta=1.0)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

best_val_loss = float("inf")

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0.0
    for x, y in train_loader:
        x, y = x.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad()
        preds = model(x)

        # Huber Loss
        huber_loss = criterion(preds, y)

        # SSIM Loss (1 - SSIM)
        ssim_loss = 1 - ssim(preds, y, data_range=1.0, size_average=True)  # assumes preds/y in [0,1]

        # Combined Loss
        loss = alpha * huber_loss + beta * ssim_loss
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * x.size(0)
    train_loss /= len(train_loader.dataset)

    # ---- Validation ----
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(DEVICE), y.to(DEVICE)
            preds = model(x)

            huber_loss = criterion(preds, y)
            ssim_loss = 1 - ssim(preds, y, data_range=1.0, size_average=True)
            loss = alpha * huber_loss + beta * ssim_loss

            val_loss += loss.item() * x.size(0)
    val_loss /= len(val_loader.dataset)

    print(f"Epoch [{epoch+1}/{EPOCHS}] Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), MODEL_PATH)
        print(f"  ✅ Saved Best Model at Epoch {epoch+1}")

# ==========================
# INFERENCE
# ==========================
print("\nRunning inference on test set...")
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.eval()

for i, (x, y) in enumerate(test_loader):
    x = x.to(DEVICE)
    with torch.no_grad():
        pred = model(x).cpu().squeeze(0).squeeze(0).numpy()
    pred_img = (pred * 255.0).clip(0, 255).astype(np.uint8)
    Image.fromarray(pred_img).save(os.path.join(RESULTS_DIR, f"pred_{i+1}.png"))

print(f"Predictions saved in {RESULTS_DIR}")

Running on: cuda
Train: 2800, Val: 600, Test: 600
Epoch [1/75] Train Loss: 0.040619, Val Loss: 0.002880
  ✅ Saved Best Model at Epoch 1
Epoch [2/75] Train Loss: 0.001793, Val Loss: 0.001128
  ✅ Saved Best Model at Epoch 2
Epoch [3/75] Train Loss: 0.000946, Val Loss: 0.000794
  ✅ Saved Best Model at Epoch 3
Epoch [4/75] Train Loss: 0.000729, Val Loss: 0.000721
  ✅ Saved Best Model at Epoch 4
Epoch [5/75] Train Loss: 0.000640, Val Loss: 0.000583
  ✅ Saved Best Model at Epoch 5
Epoch [6/75] Train Loss: 0.000576, Val Loss: 0.000658
Epoch [7/75] Train Loss: 0.000545, Val Loss: 0.000576
  ✅ Saved Best Model at Epoch 7
Epoch [8/75] Train Loss: 0.000516, Val Loss: 0.000478
  ✅ Saved Best Model at Epoch 8
Epoch [9/75] Train Loss: 0.000500, Val Loss: 0.000495
Epoch [10/75] Train Loss: 0.000480, Val Loss: 0.000476
  ✅ Saved Best Model at Epoch 10
Epoch [11/75] Train Loss: 0.000457, Val Loss: 0.000458
  ✅ Saved Best Model at Epoch 11
Epoch [12/75] Train Loss: 0.000440, Val Loss: 0.000433
  ✅ Saved

In [5]:
import os
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
from math import log10


import shutil
os.makedirs(r"C:\Preet\4000_paired_bscans\ground_truth_test_final_huber+ssim", exist_ok=True)

for f in test_y:  # list of test high-res paths from your train/val/test split
    shutil.copy(f, r"C:\Preet\4000_paired_bscans\ground_truth_test_final_huber+ssim")


# ==== PATHS ====
pred_dir = r"C:\Preet\4000_paired_bscans\predictions_attention_final_huber_ssim"
gt_dir   = r"C:\Preet\4000_paired_bscans\ground_truth_test_final_huber+ssim"

# ==== FUNCTIONS ====
def calculate_psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0:
        return float('inf')
    return 20 * log10(255.0 / np.sqrt(mse))

def calculate_ssim(img1, img2):
    return ssim(img1, img2, data_range=255)

# ==== MAIN ====
psnr_values, ssim_values = [], []

pred_files = sorted([f for f in os.listdir(pred_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
gt_files   = sorted([f for f in os.listdir(gt_dir)   if f.lower().endswith(('.png', '.jpg', '.jpeg'))])

num_pairs = min(len(pred_files), len(gt_files))
if num_pairs == 0:
    print("[Error] No matching image files found in both directories.")
else:
    if len(pred_files) != len(gt_files):
        print(f"[Warning] Different number of images: Predictions={len(pred_files)}, Ground Truth={len(gt_files)}")
        print(f"Evaluating only first {num_pairs} matched pairs.")

    for i in range(num_pairs):
        pred_path = os.path.join(pred_dir, pred_files[i])
        gt_path   = os.path.join(gt_dir, gt_files[i])

        pred_img = cv2.imread(pred_path, cv2.IMREAD_GRAYSCALE)
        gt_img   = cv2.imread(gt_path, cv2.IMREAD_GRAYSCALE)

        if pred_img is None or gt_img is None:
            print(f"[Error] Could not load: {pred_files[i]} or {gt_files[i]}")
            continue

        if pred_img.shape != gt_img.shape:
            pred_img = cv2.resize(pred_img, (gt_img.shape[1], gt_img.shape[0]))

        psnr_values.append(calculate_psnr(pred_img, gt_img))
        ssim_values.append(calculate_ssim(pred_img, gt_img))

    if psnr_values and ssim_values:
        print(f"\n---- Test Set Evaluation ----")
        print(f"SSIM: avg={np.mean(ssim_values):.4f}, min={np.min(ssim_values):.4f}, max={np.max(ssim_values):.4f}")
        print(f"PSNR: avg={np.mean(psnr_values):.2f} dB, min={np.min(psnr_values):.2f} dB, max={np.max(psnr_values):.2f} dB")


---- Test Set Evaluation ----
SSIM: avg=0.8917, min=0.7145, max=0.9982
PSNR: avg=35.44 dB, min=28.91 dB, max=53.08 dB


In [2]:
import os
import re
import numpy as np
from PIL import Image
import cv2
from math import log10
from skimage.metrics import structural_similarity as ssim

import torch
import torch.nn as nn

# =========================
# CONFIG
# =========================
FREQ_DIRS = [
    r"C:\Preet\validation dataset_4000\png_images_650M_1083M",
    r"C:\Preet\validation dataset_4000\png_images_700M_1167M",
    r"C:\Preet\validation dataset_4000\png_images_800M_1333M",
    r"C:\Preet\validation dataset_4000\png_images_850M_1416M",
    r"C:\Preet\validation dataset_4000\png_images_900M_1500M",
]

MODEL_PATH = r"C:\Preet\4000_paired_bscans\best_attention_unet_final_huber+ssim.pth"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
IMAGE_SIZE = (256, 256)

# =========================
# MODEL BLOCKS (UNet3 + 6 RB)
# =========================
class ConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.ReLU(inplace=True),
        )
    def forward(self, x): return self.conv(x)

class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(channels, channels, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),
            nn.Conv2d(channels, channels, 3, padding=1),
        )
    def forward(self, x): return x + self.conv(x)

class AttentionGate(nn.Module):
    def __init__(self, F_g, F_l, F_int):
        super().__init__()
        self.W_g = nn.Conv2d(F_g, F_int, 1)
        self.W_x = nn.Conv2d(F_l, F_int, 1)
        self.psi = nn.Sequential(
            nn.ReLU(inplace=True),
            nn.Conv2d(F_int, 1, 1),
            nn.Sigmoid()
        )
    def forward(self, x, g):
        g1 = self.W_g(g)
        x1 = self.W_x(x)
        psi = self.psi(g1 + x1)
        return x * psi

class UNet3WithAttention(nn.Module):
    def __init__(self):
        super().__init__()
        # Encoder
        self.enc1 = ConvBlock(1, 64); self.pool1 = nn.MaxPool2d(2)
        self.enc2 = ConvBlock(64, 128); self.pool2 = nn.MaxPool2d(2)
        self.enc3 = ConvBlock(128, 256); self.pool3 = nn.MaxPool2d(2)

        # Bottleneck
        self.bottleneck = nn.Sequential(*[ResidualBlock(256) for _ in range(6)])

        # Decoder
        self.up3 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.att3 = AttentionGate(256, 256, 128)
        self.dec3 = ConvBlock(512, 256)

        self.up2 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.att2 = AttentionGate(256, 128, 64)
        self.dec2 = ConvBlock(384, 128)

        self.up1 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.att1 = AttentionGate(128, 64, 32)
        self.dec1 = ConvBlock(192, 64)

        self.out_conv = nn.Conv2d(64, 1, 1)

    def forward(self, x):
        c1 = self.enc1(x); p1 = self.pool1(c1)
        c2 = self.enc2(p1); p2 = self.pool2(c2)
        c3 = self.enc3(p2); p3 = self.pool3(c3)
        b = self.bottleneck(p3)

        u3 = self.up3(b); a3 = self.att3(c3, u3); d3 = self.dec3(torch.cat([u3, a3], dim=1))
        u2 = self.up2(d3); a2 = self.att2(c2, u2); d2 = self.dec2(torch.cat([u2, a2], dim=1))
        u1 = self.up1(d2); a1 = self.att1(c1, u1); d1 = self.dec1(torch.cat([u1, a1], dim=1))
        return self.out_conv(d1)

# =========================
# HELPERS
# =========================
def calculate_psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0: return float('inf')
    return 20 * log10(255.0 / np.sqrt(mse))

def calculate_ssim(img1, img2):
    return ssim(img1, img2, data_range=255)

def numerical_sort(files):
    return sorted(files, key=lambda f: int(re.search(r'\d+', f).group()))

# =========================
# LOAD MODEL
# =========================
model = UNet3WithAttention().to(DEVICE)
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.eval()

# =========================
# INFERENCE LOOP
# =========================
for freq_dir in FREQ_DIRS:
    print(f"\n---- Evaluating {os.path.basename(freq_dir)} ----")
    pred_dir = os.path.join(freq_dir, "predictions_attention_unet_final_huber+ssim")
    os.makedirs(pred_dir, exist_ok=True)

    low_files = numerical_sort([f for f in os.listdir(freq_dir) if f.endswith("_l.png")])
    high_files = numerical_sort([f for f in os.listdir(freq_dir) if f.endswith("_h.png")])

    psnr_values, ssim_values = [], []

    for i in range(len(low_files)):
        low_path = os.path.join(freq_dir, low_files[i])
        high_path = os.path.join(freq_dir, high_files[i])

        # Load LR
        lr_img = np.array(Image.open(low_path).resize(IMAGE_SIZE), dtype=np.float32) / 255.0
        lr_tensor = torch.tensor(lr_img).unsqueeze(0).unsqueeze(0).to(DEVICE)

        # Predict
        with torch.no_grad():
            pred = model(lr_tensor).cpu().numpy()

        # remove batch and channel dims, scale to 0-255
        pred_img = np.squeeze(pred)
        pred_img = (pred_img * 255.0).clip(0, 255).astype(np.uint8)

        # increase contrast (so it's dark like GT)
        pred_img = cv2.normalize(pred_img, None, 0, 255, cv2.NORM_MINMAX)

        # save prediction
        pred_path = os.path.join(pred_dir, f"pred_{i+1}.png")
        Image.fromarray(pred_img).save(pred_path)

        # Load GT
        gt_img = cv2.imread(high_path, cv2.IMREAD_GRAYSCALE)
        if pred_img.shape != gt_img.shape:
            pred_img = cv2.resize(pred_img, (gt_img.shape[1], gt_img.shape[0]))

        # Metrics
        psnr_values.append(calculate_psnr(pred_img, gt_img))
        ssim_values.append(calculate_ssim(pred_img, gt_img))

    # Results
    if psnr_values and ssim_values:
        print(f"SSIM: avg={np.mean(ssim_values):.4f}, min={np.min(ssim_values):.4f}, max={np.max(ssim_values):.4f}")
        print(f"PSNR: avg={np.mean(psnr_values):.2f} dB, min={np.min(psnr_values):.2f} dB, max={np.max(psnr_values):.2f} dB")
    else:
        print("[Error] No valid pairs processed.")



---- Evaluating png_images_650M_1083M ----
SSIM: avg=0.9550, min=0.9236, max=0.9674
PSNR: avg=36.26 dB, min=31.81 dB, max=37.62 dB

---- Evaluating png_images_700M_1167M ----
SSIM: avg=0.9871, min=0.7610, max=0.9920
PSNR: avg=38.43 dB, min=29.30 dB, max=39.51 dB

---- Evaluating png_images_800M_1333M ----
SSIM: avg=0.9908, min=0.9758, max=0.9943
PSNR: avg=40.18 dB, min=34.67 dB, max=41.73 dB

---- Evaluating png_images_850M_1416M ----
SSIM: avg=0.9559, min=0.9182, max=0.9661
PSNR: avg=33.57 dB, min=31.07 dB, max=34.71 dB

---- Evaluating png_images_900M_1500M ----
SSIM: avg=0.9275, min=0.8798, max=0.9461
PSNR: avg=32.49 dB, min=29.60 dB, max=33.56 dB


In [3]:
import os
import re
import numpy as np
from PIL import Image
import cv2
from math import log10
from skimage.metrics import structural_similarity as ssim

import torch
import torch.nn as nn

# =========================
# CONFIG
# =========================
DATASET_DIR = r"C:\Preet\validation dataset_4000\png_images_650M_1083M_1800M"
MODEL_PATH = r"C:\Preet\4000_paired_bscans\best_attention_unet_final_huber+ssim.pth"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
IMAGE_SIZE = (256, 256)

# =========================
# MODEL BLOCKS
# =========================
class ConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.ReLU(inplace=True),
        )
    def forward(self, x): return self.conv(x)

class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(channels, channels, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),
            nn.Conv2d(channels, channels, 3, padding=1),
        )
    def forward(self, x): return x + self.conv(x)

class AttentionGate(nn.Module):
    def __init__(self, F_g, F_l, F_int):
        super().__init__()
        self.W_g = nn.Conv2d(F_g, F_int, 1)
        self.W_x = nn.Conv2d(F_l, F_int, 1)
        self.psi = nn.Sequential(
            nn.ReLU(inplace=True),
            nn.Conv2d(F_int, 1, 1),
            nn.Sigmoid()
        )
    def forward(self, x, g):
        g1 = self.W_g(g)
        x1 = self.W_x(x)
        psi = self.psi(g1 + x1)
        return x * psi

class UNet3WithAttention(nn.Module):
    def __init__(self):
        super().__init__()
        # Encoder
        self.enc1 = ConvBlock(1, 64); self.pool1 = nn.MaxPool2d(2)
        self.enc2 = ConvBlock(64, 128); self.pool2 = nn.MaxPool2d(2)
        self.enc3 = ConvBlock(128, 256); self.pool3 = nn.MaxPool2d(2)

        # Bottleneck
        self.bottleneck = nn.Sequential(*[ResidualBlock(256) for _ in range(6)])

        # Decoder
        self.up3 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.att3 = AttentionGate(256, 256, 128)
        self.dec3 = ConvBlock(512, 256)

        self.up2 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.att2 = AttentionGate(256, 128, 64)
        self.dec2 = ConvBlock(384, 128)

        self.up1 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.att1 = AttentionGate(128, 64, 32)
        self.dec1 = ConvBlock(192, 64)

        self.out_conv = nn.Conv2d(64, 1, 1)

    def forward(self, x):
        c1 = self.enc1(x); p1 = self.pool1(c1)
        c2 = self.enc2(p1); p2 = self.pool2(c2)
        c3 = self.enc3(p2); p3 = self.pool3(c3)
        b = self.bottleneck(p3)

        u3 = self.up3(b); a3 = self.att3(c3, u3); d3 = self.dec3(torch.cat([u3, a3], dim=1))
        u2 = self.up2(d3); a2 = self.att2(c2, u2); d2 = self.dec2(torch.cat([u2, a2], dim=1))
        u1 = self.up1(d2); a1 = self.att1(c1, u1); d1 = self.dec1(torch.cat([u1, a1], dim=1))
        return self.out_conv(d1)

# =========================
# HELPERS
# =========================
def calculate_psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0: return float('inf')
    return 20 * log10(255.0 / np.sqrt(mse))

def calculate_ssim(img1, img2):
    return ssim(img1, img2, data_range=255)

def numerical_sort(files):
    return sorted(files, key=lambda f: int(re.search(r'\d+', f).group()))

def run_model(img, model):
    tensor = torch.tensor(img, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(DEVICE)
    with torch.no_grad():
        pred = model(tensor).cpu().numpy()
    pred = np.squeeze(pred)
    pred = (pred * 255.0).clip(0, 255).astype(np.uint8)
    return cv2.normalize(pred, None, 0, 255, cv2.NORM_MINMAX)

# =========================
# LOAD MODEL
# =========================
model = UNet3WithAttention().to(DEVICE)
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.eval()

# =========================
# INFERENCE 2-STAGE
# =========================
files_650 = numerical_sort([f for f in os.listdir(DATASET_DIR) if f.endswith("650_bscan.png")])
files_1083 = numerical_sort([f for f in os.listdir(DATASET_DIR) if f.endswith("1083_bscan.png")])
files_1800 = numerical_sort([f for f in os.listdir(DATASET_DIR) if f.endswith("1800_bscan.png")])

psnr_stage1, ssim_stage1 = [], []
psnr_stage2, ssim_stage2 = [], []

pred_dir1 = os.path.join(DATASET_DIR, "predictions_stage1_huber+loss")
pred_dir2 = os.path.join(DATASET_DIR, "predictions_stage2_huber+loss")
os.makedirs(pred_dir1, exist_ok=True)
os.makedirs(pred_dir2, exist_ok=True)

for i in range(len(files_650)):
    # ---- Stage 1: 650 -> pred -> compare with 1083 ----
    lr_img = np.array(Image.open(os.path.join(DATASET_DIR, files_650[i])).resize(IMAGE_SIZE), dtype=np.float32) / 255.0
    gt_1083 = cv2.imread(os.path.join(DATASET_DIR, files_1083[i]), cv2.IMREAD_GRAYSCALE)

    pred_1083 = run_model(lr_img, model)
    Image.fromarray(pred_1083).save(os.path.join(pred_dir1, f"pred1_{i+1}.png"))

    if pred_1083.shape != gt_1083.shape:
        pred_1083 = cv2.resize(pred_1083, (gt_1083.shape[1], gt_1083.shape[0]))

    psnr_stage1.append(calculate_psnr(pred_1083, gt_1083))
    ssim_stage1.append(calculate_ssim(pred_1083, gt_1083))

    # ---- Stage 2: pred_1083 -> pred -> compare with 1800 ----
    gt_1800 = cv2.imread(os.path.join(DATASET_DIR, files_1800[i]), cv2.IMREAD_GRAYSCALE)

    # resize pred_1083 to IMAGE_SIZE before refeeding
    pred_1083_resized = cv2.resize(pred_1083, IMAGE_SIZE).astype(np.float32) / 255.0
    pred_1800 = run_model(pred_1083_resized, model)
    Image.fromarray(pred_1800).save(os.path.join(pred_dir2, f"pred2_{i+1}.png"))

    if pred_1800.shape != gt_1800.shape:
        pred_1800 = cv2.resize(pred_1800, (gt_1800.shape[1], gt_1800.shape[0]))

    psnr_stage2.append(calculate_psnr(pred_1800, gt_1800))
    ssim_stage2.append(calculate_ssim(pred_1800, gt_1800))

# =========================
# RESULTS
# =========================
print("\n---- Stage 1: 650 → pred → compare with 1083 ----")
print(f"SSIM: avg={np.mean(ssim_stage1):.4f}, min={np.min(ssim_stage1):.4f}, max={np.max(ssim_stage1):.4f}")
print(f"PSNR: avg={np.mean(psnr_stage1):.2f} dB, min={np.min(psnr_stage1):.2f} dB, max={np.max(psnr_stage1):.2f} dB")

print("\n---- Stage 2: pred_1083 → pred → compare with 1800 ----")
print(f"SSIM: avg={np.mean(ssim_stage2):.4f}, min={np.min(ssim_stage2):.4f}, max={np.max(ssim_stage2):.4f}")
print(f"PSNR: avg={np.mean(psnr_stage2):.2f} dB, min={np.min(psnr_stage2):.2f} dB, max={np.max(psnr_stage2):.2f} dB")



---- Stage 1: 650 → pred → compare with 1083 ----
SSIM: avg=0.9528, min=0.9263, max=0.9668
PSNR: avg=36.07 dB, min=32.43 dB, max=37.59 dB

---- Stage 2: pred_1083 → pred → compare with 1800 ----
SSIM: avg=0.9299, min=0.8887, max=0.9489
PSNR: avg=28.92 dB, min=28.06 dB, max=30.14 dB


In [4]:
import os
import re
import numpy as np
from PIL import Image
import cv2
from math import log10
from skimage.metrics import structural_similarity as ssim

import torch
import torch.nn as nn

# =========================
# CONFIG
# =========================
DATASET_DIR = r"C:\Preet\validation dataset_4000\png_images_900M_1500M_2500M"
MODEL_PATH = r"C:\Preet\4000_paired_bscans\best_attention_unet_final_huber+ssim.pth"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
IMAGE_SIZE = (256, 256)

# =========================
# MODEL BLOCKS
# =========================
class ConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.ReLU(inplace=True),
        )
    def forward(self, x): return self.conv(x)

class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(channels, channels, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),
            nn.Conv2d(channels, channels, 3, padding=1),
        )
    def forward(self, x): return x + self.conv(x)

class AttentionGate(nn.Module):
    def __init__(self, F_g, F_l, F_int):
        super().__init__()
        self.W_g = nn.Conv2d(F_g, F_int, 1)
        self.W_x = nn.Conv2d(F_l, F_int, 1)
        self.psi = nn.Sequential(
            nn.ReLU(inplace=True),
            nn.Conv2d(F_int, 1, 1),
            nn.Sigmoid()
        )
    def forward(self, x, g):
        g1 = self.W_g(g)
        x1 = self.W_x(x)
        psi = self.psi(g1 + x1)
        return x * psi

class UNet3WithAttention(nn.Module):
    def __init__(self):
        super().__init__()
        # Encoder
        self.enc1 = ConvBlock(1, 64); self.pool1 = nn.MaxPool2d(2)
        self.enc2 = ConvBlock(64, 128); self.pool2 = nn.MaxPool2d(2)
        self.enc3 = ConvBlock(128, 256); self.pool3 = nn.MaxPool2d(2)

        # Bottleneck
        self.bottleneck = nn.Sequential(*[ResidualBlock(256) for _ in range(6)])

        # Decoder
        self.up3 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.att3 = AttentionGate(256, 256, 128)
        self.dec3 = ConvBlock(512, 256)

        self.up2 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.att2 = AttentionGate(256, 128, 64)
        self.dec2 = ConvBlock(384, 128)

        self.up1 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.att1 = AttentionGate(128, 64, 32)
        self.dec1 = ConvBlock(192, 64)

        self.out_conv = nn.Conv2d(64, 1, 1)

    def forward(self, x):
        c1 = self.enc1(x); p1 = self.pool1(c1)
        c2 = self.enc2(p1); p2 = self.pool2(c2)
        c3 = self.enc3(p2); p3 = self.pool3(c3)
        b = self.bottleneck(p3)

        u3 = self.up3(b); a3 = self.att3(c3, u3); d3 = self.dec3(torch.cat([u3, a3], dim=1))
        u2 = self.up2(d3); a2 = self.att2(c2, u2); d2 = self.dec2(torch.cat([u2, a2], dim=1))
        u1 = self.up1(d2); a1 = self.att1(c1, u1); d1 = self.dec1(torch.cat([u1, a1], dim=1))
        return self.out_conv(d1)

# =========================
# HELPERS
# =========================
def calculate_psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0: return float('inf')
    return 20 * log10(255.0 / np.sqrt(mse))

def calculate_ssim(img1, img2):
    return ssim(img1, img2, data_range=255)

def numerical_sort(files):
    return sorted(files, key=lambda f: int(re.search(r'\d+', f).group()))

def run_model(img, model):
    tensor = torch.tensor(img, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(DEVICE)
    with torch.no_grad():
        pred = model(tensor).cpu().numpy()
    pred = np.squeeze(pred)
    pred = (pred * 255.0).clip(0, 255).astype(np.uint8)
    return cv2.normalize(pred, None, 0, 255, cv2.NORM_MINMAX)

# =========================
# LOAD MODEL
# =========================
model = UNet3WithAttention().to(DEVICE)
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.eval()

# =========================
# INFERENCE 2-STAGE
# =========================
files_650 = numerical_sort([f for f in os.listdir(DATASET_DIR) if f.endswith("900_bscan.png")])
files_1083 = numerical_sort([f for f in os.listdir(DATASET_DIR) if f.endswith("1500_bscan.png")])
files_1800 = numerical_sort([f for f in os.listdir(DATASET_DIR) if f.endswith("2500_bscan.png")])

psnr_stage1, ssim_stage1 = [], []
psnr_stage2, ssim_stage2 = [], []

pred_dir1 = os.path.join(DATASET_DIR, "predictions_stage1_huber+loss")
pred_dir2 = os.path.join(DATASET_DIR, "predictions_stage2_huber+loss")
os.makedirs(pred_dir1, exist_ok=True)
os.makedirs(pred_dir2, exist_ok=True)

for i in range(len(files_650)):
    # ---- Stage 1: 650 -> pred -> compare with 1083 ----
    lr_img = np.array(Image.open(os.path.join(DATASET_DIR, files_650[i])).resize(IMAGE_SIZE), dtype=np.float32) / 255.0
    gt_1083 = cv2.imread(os.path.join(DATASET_DIR, files_1083[i]), cv2.IMREAD_GRAYSCALE)

    pred_1083 = run_model(lr_img, model)
    Image.fromarray(pred_1083).save(os.path.join(pred_dir1, f"pred1_{i+1}.png"))

    if pred_1083.shape != gt_1083.shape:
        pred_1083 = cv2.resize(pred_1083, (gt_1083.shape[1], gt_1083.shape[0]))

    psnr_stage1.append(calculate_psnr(pred_1083, gt_1083))
    ssim_stage1.append(calculate_ssim(pred_1083, gt_1083))

    # ---- Stage 2: pred_1083 -> pred -> compare with 1800 ----
    gt_1800 = cv2.imread(os.path.join(DATASET_DIR, files_1800[i]), cv2.IMREAD_GRAYSCALE)

    # resize pred_1083 to IMAGE_SIZE before refeeding
    pred_1083_resized = cv2.resize(pred_1083, IMAGE_SIZE).astype(np.float32) / 255.0
    pred_1800 = run_model(pred_1083_resized, model)
    Image.fromarray(pred_1800).save(os.path.join(pred_dir2, f"pred2_{i+1}.png"))

    if pred_1800.shape != gt_1800.shape:
        pred_1800 = cv2.resize(pred_1800, (gt_1800.shape[1], gt_1800.shape[0]))

    psnr_stage2.append(calculate_psnr(pred_1800, gt_1800))
    ssim_stage2.append(calculate_ssim(pred_1800, gt_1800))

# =========================
# RESULTS
# =========================
print("\n---- Stage 1: 900 → pred → compare with 1500 ----")
print(f"SSIM: avg={np.mean(ssim_stage1):.4f}, min={np.min(ssim_stage1):.4f}, max={np.max(ssim_stage1):.4f}")
print(f"PSNR: avg={np.mean(psnr_stage1):.2f} dB, min={np.min(psnr_stage1):.2f} dB, max={np.max(psnr_stage1):.2f} dB")

print("\n---- Stage 2: pred_1500 → pred → compare with 2500 ----")
print(f"SSIM: avg={np.mean(ssim_stage2):.4f}, min={np.min(ssim_stage2):.4f}, max={np.max(ssim_stage2):.4f}")
print(f"PSNR: avg={np.mean(psnr_stage2):.2f} dB, min={np.min(psnr_stage2):.2f} dB, max={np.max(psnr_stage2):.2f} dB")


---- Stage 1: 900 → pred → compare with 1500 ----
SSIM: avg=0.7233, min=0.5901, max=0.8575
PSNR: avg=28.64 dB, min=28.00 dB, max=29.25 dB

---- Stage 2: pred_1500 → pred → compare with 2500 ----
SSIM: avg=0.7060, min=0.6123, max=0.8212
PSNR: avg=27.41 dB, min=26.94 dB, max=27.72 dB


----------------------

-----------

----------

-----------

clean dataset

In [1]:
import os
import cv2
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

# ==========================
# CONFIG
# ==========================
DATASET_DIR = r"C:\Preet\clean_paired_bscans"  # both *_l.png and *_h.png are here
IMAGE_SIZE = (256, 256)
BATCH_SIZE = 8
EPOCHS = 75
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Running on:", DEVICE)

MODEL_PATH = os.path.join(DATASET_DIR, "best_attention_unet_final_huber+ssim.pth")
RESULTS_DIR = os.path.join(DATASET_DIR, "predictions_attention_final_huber_ssim")
os.makedirs(RESULTS_DIR, exist_ok=True)

# ==========================
# DATASET
# ==========================
class GPRDataset(Dataset):
    def __init__(self, x_paths, y_paths):
        self.x_paths = x_paths
        self.y_paths = y_paths

    def __len__(self):
        return len(self.x_paths)

    def __getitem__(self, idx):
        x = np.array(Image.open(self.x_paths[idx]).resize(IMAGE_SIZE), dtype=np.float32) / 255.0
        y = np.array(Image.open(self.y_paths[idx]).resize(IMAGE_SIZE), dtype=np.float32) / 255.0
        x = torch.tensor(x).unsqueeze(0)  # (1,H,W)
        y = torch.tensor(y).unsqueeze(0)
        return x, y

def load_data(dataset_dir):
    low_paths, high_paths = [], []
    for file in os.listdir(dataset_dir):
        if file.endswith("_l.png"):
            low_path = os.path.join(dataset_dir, file)
            high_path = os.path.join(dataset_dir, file.replace("_l.png", "_h.png"))
            if os.path.exists(high_path):
                low_paths.append(low_path)
                high_paths.append(high_path)
    return low_paths, high_paths

all_x, all_y = load_data(DATASET_DIR)

# Split 70/15/15
train_x, temp_x, train_y, temp_y = train_test_split(all_x, all_y, test_size=0.30, random_state=42)
val_x, test_x, val_y, test_y = train_test_split(temp_x, temp_y, test_size=0.50, random_state=42)

print(f"Train: {len(train_x)}, Val: {len(val_x)}, Test: {len(test_x)}")

train_loader = DataLoader(GPRDataset(train_x, train_y), batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(GPRDataset(val_x, val_y), batch_size=BATCH_SIZE, shuffle=False)
test_loader  = DataLoader(GPRDataset(test_x, test_y), batch_size=1, shuffle=False)

# ==========================
# MODEL BLOCKS
# ==========================
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, padding=1),
            nn.ReLU(inplace=True),
        )
    def forward(self, x):
        return self.conv(x)

class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(channels, channels, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),
            nn.Conv2d(channels, channels, 3, padding=1),
        )
    def forward(self, x):
        return x + self.conv(x)

class AttentionGate(nn.Module):
    def __init__(self, F_g, F_l, F_int):
        super().__init__()
        self.W_g = nn.Conv2d(F_g, F_int, kernel_size=1)
        self.W_x = nn.Conv2d(F_l, F_int, kernel_size=1)
        self.psi = nn.Sequential(
            nn.ReLU(inplace=True),
            nn.Conv2d(F_int, 1, kernel_size=1),
            nn.Sigmoid()
        )
    def forward(self, x, g):
        g1 = self.W_g(g)
        x1 = self.W_x(x)
        psi = self.psi(g1 + x1)
        return x * psi

# ==========================
# UNet-3 with Attention + Residual Bottleneck
# ==========================
class UNet3WithAttention(nn.Module):
    def __init__(self):
        super().__init__()
        # Encoder (3 levels)
        self.enc1 = ConvBlock(1, 64); self.pool1 = nn.MaxPool2d(2)
        self.enc2 = ConvBlock(64, 128); self.pool2 = nn.MaxPool2d(2)
        self.enc3 = ConvBlock(128, 256); self.pool3 = nn.MaxPool2d(2)

        # Bottleneck with residual blocks (FIX: 256 channels)
        self.bottleneck = nn.Sequential(*[ResidualBlock(256) for _ in range(6)])

        # Decoder
        self.up3 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.att3 = AttentionGate(256, 256, 128)   # bottleneck=256, skip=256
        self.dec3 = ConvBlock(256+256, 256)        # concat → 512

        self.up2 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.att2 = AttentionGate(256, 128, 64)
        self.dec2 = ConvBlock(256+128, 128)

        self.up1 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.att1 = AttentionGate(128, 64, 32)
        self.dec1 = ConvBlock(128+64, 64)

        self.out_conv = nn.Conv2d(64, 1, kernel_size=1)

    def forward(self, x):
        # Encoder
        c1 = self.enc1(x); p1 = self.pool1(c1)
        c2 = self.enc2(p1); p2 = self.pool2(c2)
        c3 = self.enc3(p2); p3 = self.pool3(c3)

        # Bottleneck
        b = self.bottleneck(p3)

        # Decoder
        u3 = self.up3(b); a3 = self.att3(c3, u3); d3 = self.dec3(torch.cat([u3, a3], dim=1))
        u2 = self.up2(d3); a2 = self.att2(c2, u2); d2 = self.dec2(torch.cat([u2, a2], dim=1))
        u1 = self.up1(d2); a1 = self.att1(c1, u1); d1 = self.dec1(torch.cat([u1, a1], dim=1))

        return self.out_conv(d1)


# ==========================
# TRAINING
# ==========================
model = UNet3WithAttention().to(DEVICE)
from pytorch_msssim import ssim  # pip install pytorch-msssim

alpha = 0.8
beta  = 0.2

criterion = nn.HuberLoss(delta=1.0)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

best_val_loss = float("inf")

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0.0
    for x, y in train_loader:
        x, y = x.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad()
        preds = model(x)

        # Huber Loss
        huber_loss = criterion(preds, y)

        # SSIM Loss (1 - SSIM)
        ssim_loss = 1 - ssim(preds, y, data_range=1.0, size_average=True)  # assumes preds/y in [0,1]

        # Combined Loss
        loss = alpha * huber_loss + beta * ssim_loss
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * x.size(0)
    train_loss /= len(train_loader.dataset)

    # ---- Validation ----
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(DEVICE), y.to(DEVICE)
            preds = model(x)

            huber_loss = criterion(preds, y)
            ssim_loss = 1 - ssim(preds, y, data_range=1.0, size_average=True)
            loss = alpha * huber_loss + beta * ssim_loss

            val_loss += loss.item() * x.size(0)
    val_loss /= len(val_loader.dataset)

    print(f"Epoch [{epoch+1}/{EPOCHS}] Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), MODEL_PATH)
        print(f"  ✅ Saved Best Model at Epoch {epoch+1}")

# ==========================
# INFERENCE
# ==========================
print("\nRunning inference on test set...")
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.eval()

for i, (x, y) in enumerate(test_loader):
    x = x.to(DEVICE)
    with torch.no_grad():
        pred = model(x).cpu().squeeze(0).squeeze(0).numpy()
    pred_img = (pred * 255.0).clip(0, 255).astype(np.uint8)
    Image.fromarray(pred_img).save(os.path.join(RESULTS_DIR, f"pred_{i+1}.png"))

print(f"Predictions saved in {RESULTS_DIR}")

Running on: cuda
Train: 5810, Val: 1245, Test: 1245
Epoch [1/75] Train Loss: 0.010355, Val Loss: 0.000591
  ✅ Saved Best Model at Epoch 1
Epoch [2/75] Train Loss: 0.000495, Val Loss: 0.000757
Epoch [3/75] Train Loss: 0.000405, Val Loss: 0.000447
  ✅ Saved Best Model at Epoch 3
Epoch [4/75] Train Loss: 0.000355, Val Loss: 0.000349
  ✅ Saved Best Model at Epoch 4
Epoch [5/75] Train Loss: 0.000329, Val Loss: 0.000330
  ✅ Saved Best Model at Epoch 5
Epoch [6/75] Train Loss: 0.000310, Val Loss: 0.000337
Epoch [7/75] Train Loss: 0.000294, Val Loss: 0.000325
  ✅ Saved Best Model at Epoch 7
Epoch [8/75] Train Loss: 0.000278, Val Loss: 0.000279
  ✅ Saved Best Model at Epoch 8
Epoch [9/75] Train Loss: 0.000265, Val Loss: 0.000253
  ✅ Saved Best Model at Epoch 9
Epoch [10/75] Train Loss: 0.000249, Val Loss: 0.000244
  ✅ Saved Best Model at Epoch 10
Epoch [11/75] Train Loss: 0.000241, Val Loss: 0.000220
  ✅ Saved Best Model at Epoch 11
Epoch [12/75] Train Loss: 0.000233, Val Loss: 0.000228
Epoch [

In [2]:
import os
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
from math import log10


import shutil
os.makedirs(r"C:\Preet\clean_paired_bscans\ground_truth_test_final_huber+ssim", exist_ok=True)

for f in test_y:  # list of test high-res paths from your train/val/test split
    shutil.copy(f, r"C:\Preet\clean_paired_bscans\ground_truth_test_final_huber+ssim")


# ==== PATHS ====
pred_dir = r"C:\Preet\clean_paired_bscans\predictions_attention_final_huber_ssim"
gt_dir   = r"C:\Preet\clean_paired_bscans\ground_truth_test_final_huber+ssim"

# ==== FUNCTIONS ====
def calculate_psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0:
        return float('inf')
    return 20 * log10(255.0 / np.sqrt(mse))

def calculate_ssim(img1, img2):
    return ssim(img1, img2, data_range=255)

# ==== MAIN ====
psnr_values, ssim_values = [], []

pred_files = sorted([f for f in os.listdir(pred_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
gt_files   = sorted([f for f in os.listdir(gt_dir)   if f.lower().endswith(('.png', '.jpg', '.jpeg'))])

num_pairs = min(len(pred_files), len(gt_files))
if num_pairs == 0:
    print("[Error] No matching image files found in both directories.")
else:
    if len(pred_files) != len(gt_files):
        print(f"[Warning] Different number of images: Predictions={len(pred_files)}, Ground Truth={len(gt_files)}")
        print(f"Evaluating only first {num_pairs} matched pairs.")

    for i in range(num_pairs):
        pred_path = os.path.join(pred_dir, pred_files[i])
        gt_path   = os.path.join(gt_dir, gt_files[i])

        pred_img = cv2.imread(pred_path, cv2.IMREAD_GRAYSCALE)
        gt_img   = cv2.imread(gt_path, cv2.IMREAD_GRAYSCALE)

        if pred_img is None or gt_img is None:
            print(f"[Error] Could not load: {pred_files[i]} or {gt_files[i]}")
            continue

        if pred_img.shape != gt_img.shape:
            pred_img = cv2.resize(pred_img, (gt_img.shape[1], gt_img.shape[0]))

        psnr_values.append(calculate_psnr(pred_img, gt_img))
        ssim_values.append(calculate_ssim(pred_img, gt_img))

    if psnr_values and ssim_values:
        print(f"\n---- Test Set Evaluation ----")
        print(f"SSIM: avg={np.mean(ssim_values):.4f}, min={np.min(ssim_values):.4f}, max={np.max(ssim_values):.4f}")
        print(f"PSNR: avg={np.mean(psnr_values):.2f} dB, min={np.min(psnr_values):.2f} dB, max={np.max(psnr_values):.2f} dB")


---- Test Set Evaluation ----
SSIM: avg=0.9048, min=0.7491, max=0.9980
PSNR: avg=35.99 dB, min=29.50 dB, max=51.91 dB


In [3]:
import os
import re
import numpy as np
from PIL import Image
import cv2
from math import log10
from skimage.metrics import structural_similarity as ssim

import torch
import torch.nn as nn

# =========================
# CONFIG
# =========================
FREQ_DIRS = [
    r"C:\Preet\clean_validation dataset\png_images_650M_1083M",
    r"C:\Preet\clean_validation dataset\png_images_700M_1167M",
    r"C:\Preet\clean_validation dataset\png_images_800M_1333M",
    r"C:\Preet\clean_validation dataset\png_images_850M_1416M",
    r"C:\Preet\clean_validation dataset\png_images_900M_1500M",
]

MODEL_PATH = r"C:\Preet\clean_paired_bscans\best_attention_unet_final_huber+ssim.pth"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
IMAGE_SIZE = (256, 256)

# =========================
# MODEL BLOCKS (UNet3 + 6 RB)
# =========================
class ConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.ReLU(inplace=True),
        )
    def forward(self, x): return self.conv(x)

class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(channels, channels, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),
            nn.Conv2d(channels, channels, 3, padding=1),
        )
    def forward(self, x): return x + self.conv(x)

class AttentionGate(nn.Module):
    def __init__(self, F_g, F_l, F_int):
        super().__init__()
        self.W_g = nn.Conv2d(F_g, F_int, 1)
        self.W_x = nn.Conv2d(F_l, F_int, 1)
        self.psi = nn.Sequential(
            nn.ReLU(inplace=True),
            nn.Conv2d(F_int, 1, 1),
            nn.Sigmoid()
        )
    def forward(self, x, g):
        g1 = self.W_g(g)
        x1 = self.W_x(x)
        psi = self.psi(g1 + x1)
        return x * psi

class UNet3WithAttention(nn.Module):
    def __init__(self):
        super().__init__()
        # Encoder
        self.enc1 = ConvBlock(1, 64); self.pool1 = nn.MaxPool2d(2)
        self.enc2 = ConvBlock(64, 128); self.pool2 = nn.MaxPool2d(2)
        self.enc3 = ConvBlock(128, 256); self.pool3 = nn.MaxPool2d(2)

        # Bottleneck
        self.bottleneck = nn.Sequential(*[ResidualBlock(256) for _ in range(6)])

        # Decoder
        self.up3 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.att3 = AttentionGate(256, 256, 128)
        self.dec3 = ConvBlock(512, 256)

        self.up2 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.att2 = AttentionGate(256, 128, 64)
        self.dec2 = ConvBlock(384, 128)

        self.up1 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.att1 = AttentionGate(128, 64, 32)
        self.dec1 = ConvBlock(192, 64)

        self.out_conv = nn.Conv2d(64, 1, 1)

    def forward(self, x):
        c1 = self.enc1(x); p1 = self.pool1(c1)
        c2 = self.enc2(p1); p2 = self.pool2(c2)
        c3 = self.enc3(p2); p3 = self.pool3(c3)
        b = self.bottleneck(p3)

        u3 = self.up3(b); a3 = self.att3(c3, u3); d3 = self.dec3(torch.cat([u3, a3], dim=1))
        u2 = self.up2(d3); a2 = self.att2(c2, u2); d2 = self.dec2(torch.cat([u2, a2], dim=1))
        u1 = self.up1(d2); a1 = self.att1(c1, u1); d1 = self.dec1(torch.cat([u1, a1], dim=1))
        return self.out_conv(d1)

# =========================
# HELPERS
# =========================
def calculate_psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0: return float('inf')
    return 20 * log10(255.0 / np.sqrt(mse))

def calculate_ssim(img1, img2):
    return ssim(img1, img2, data_range=255)

def numerical_sort(files):
    return sorted(files, key=lambda f: int(re.search(r'\d+', f).group()))

# =========================
# LOAD MODEL
# =========================
model = UNet3WithAttention().to(DEVICE)
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.eval()

# =========================
# INFERENCE LOOP
# =========================
for freq_dir in FREQ_DIRS:
    print(f"\n---- Evaluating {os.path.basename(freq_dir)} ----")
    pred_dir = os.path.join(freq_dir, "predictions_attention_unet_final_huber+ssim")
    os.makedirs(pred_dir, exist_ok=True)

    low_files = numerical_sort([f for f in os.listdir(freq_dir) if f.endswith("_l.png")])
    high_files = numerical_sort([f for f in os.listdir(freq_dir) if f.endswith("_h.png")])

    psnr_values, ssim_values = [], []

    for i in range(len(low_files)):
        low_path = os.path.join(freq_dir, low_files[i])
        high_path = os.path.join(freq_dir, high_files[i])

        # Load LR
        lr_img = np.array(Image.open(low_path).resize(IMAGE_SIZE), dtype=np.float32) / 255.0
        lr_tensor = torch.tensor(lr_img).unsqueeze(0).unsqueeze(0).to(DEVICE)

        # Predict
        with torch.no_grad():
            pred = model(lr_tensor).cpu().numpy()

        # remove batch and channel dims, scale to 0-255
        pred_img = np.squeeze(pred)
        pred_img = (pred_img * 255.0).clip(0, 255).astype(np.uint8)

        # increase contrast (so it's dark like GT)
        pred_img = cv2.normalize(pred_img, None, 0, 255, cv2.NORM_MINMAX)

        # save prediction
        pred_path = os.path.join(pred_dir, f"pred_{i+1}.png")
        Image.fromarray(pred_img).save(pred_path)

        # Load GT
        gt_img = cv2.imread(high_path, cv2.IMREAD_GRAYSCALE)
        if pred_img.shape != gt_img.shape:
            pred_img = cv2.resize(pred_img, (gt_img.shape[1], gt_img.shape[0]))

        # Metrics
        psnr_values.append(calculate_psnr(pred_img, gt_img))
        ssim_values.append(calculate_ssim(pred_img, gt_img))

    # Results
    if psnr_values and ssim_values:
        print(f"SSIM: avg={np.mean(ssim_values):.4f}, min={np.min(ssim_values):.4f}, max={np.max(ssim_values):.4f}")
        print(f"PSNR: avg={np.mean(psnr_values):.2f} dB, min={np.min(psnr_values):.2f} dB, max={np.max(psnr_values):.2f} dB")
    else:
        print("[Error] No valid pairs processed.")



---- Evaluating png_images_650M_1083M ----
SSIM: avg=0.9577, min=0.9267, max=0.9689
PSNR: avg=35.16 dB, min=31.79 dB, max=36.20 dB

---- Evaluating png_images_700M_1167M ----
SSIM: avg=0.9881, min=0.8325, max=0.9920
PSNR: avg=36.94 dB, min=32.46 dB, max=37.43 dB

---- Evaluating png_images_800M_1333M ----
SSIM: avg=0.9894, min=0.9823, max=0.9924
PSNR: avg=40.06 dB, min=36.17 dB, max=41.16 dB

---- Evaluating png_images_850M_1416M ----
SSIM: avg=0.9636, min=0.9434, max=0.9733
PSNR: avg=34.14 dB, min=32.11 dB, max=35.08 dB

---- Evaluating png_images_900M_1500M ----
SSIM: avg=0.9340, min=0.8809, max=0.9509
PSNR: avg=34.75 dB, min=31.21 dB, max=36.07 dB


In [4]:
import os
import re
import numpy as np
from PIL import Image
import cv2
from math import log10
from skimage.metrics import structural_similarity as ssim

import torch
import torch.nn as nn

# =========================
# CONFIG
# =========================
DATASET_DIR = r"C:\Preet\clean_validation dataset\png_images_650M_1083M_1800M"
MODEL_PATH = r"C:\Preet\clean_paired_bscans\best_attention_unet_final_huber+ssim.pth"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
IMAGE_SIZE = (256, 256)

# =========================
# MODEL BLOCKS
# =========================
class ConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.ReLU(inplace=True),
        )
    def forward(self, x): return self.conv(x)

class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(channels, channels, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),
            nn.Conv2d(channels, channels, 3, padding=1),
        )
    def forward(self, x): return x + self.conv(x)

class AttentionGate(nn.Module):
    def __init__(self, F_g, F_l, F_int):
        super().__init__()
        self.W_g = nn.Conv2d(F_g, F_int, 1)
        self.W_x = nn.Conv2d(F_l, F_int, 1)
        self.psi = nn.Sequential(
            nn.ReLU(inplace=True),
            nn.Conv2d(F_int, 1, 1),
            nn.Sigmoid()
        )
    def forward(self, x, g):
        g1 = self.W_g(g)
        x1 = self.W_x(x)
        psi = self.psi(g1 + x1)
        return x * psi

class UNet3WithAttention(nn.Module):
    def __init__(self):
        super().__init__()
        # Encoder
        self.enc1 = ConvBlock(1, 64); self.pool1 = nn.MaxPool2d(2)
        self.enc2 = ConvBlock(64, 128); self.pool2 = nn.MaxPool2d(2)
        self.enc3 = ConvBlock(128, 256); self.pool3 = nn.MaxPool2d(2)

        # Bottleneck
        self.bottleneck = nn.Sequential(*[ResidualBlock(256) for _ in range(6)])

        # Decoder
        self.up3 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.att3 = AttentionGate(256, 256, 128)
        self.dec3 = ConvBlock(512, 256)

        self.up2 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.att2 = AttentionGate(256, 128, 64)
        self.dec2 = ConvBlock(384, 128)

        self.up1 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.att1 = AttentionGate(128, 64, 32)
        self.dec1 = ConvBlock(192, 64)

        self.out_conv = nn.Conv2d(64, 1, 1)

    def forward(self, x):
        c1 = self.enc1(x); p1 = self.pool1(c1)
        c2 = self.enc2(p1); p2 = self.pool2(c2)
        c3 = self.enc3(p2); p3 = self.pool3(c3)
        b = self.bottleneck(p3)

        u3 = self.up3(b); a3 = self.att3(c3, u3); d3 = self.dec3(torch.cat([u3, a3], dim=1))
        u2 = self.up2(d3); a2 = self.att2(c2, u2); d2 = self.dec2(torch.cat([u2, a2], dim=1))
        u1 = self.up1(d2); a1 = self.att1(c1, u1); d1 = self.dec1(torch.cat([u1, a1], dim=1))
        return self.out_conv(d1)

# =========================
# HELPERS
# =========================
def calculate_psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0: return float('inf')
    return 20 * log10(255.0 / np.sqrt(mse))

def calculate_ssim(img1, img2):
    return ssim(img1, img2, data_range=255)

def numerical_sort(files):
    return sorted(files, key=lambda f: int(re.search(r'\d+', f).group()))

def run_model(img, model):
    tensor = torch.tensor(img, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(DEVICE)
    with torch.no_grad():
        pred = model(tensor).cpu().numpy()
    pred = np.squeeze(pred)
    pred = (pred * 255.0).clip(0, 255).astype(np.uint8)
    return cv2.normalize(pred, None, 0, 255, cv2.NORM_MINMAX)

# =========================
# LOAD MODEL
# =========================
model = UNet3WithAttention().to(DEVICE)
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.eval()

# =========================
# INFERENCE 2-STAGE
# =========================
files_650 = numerical_sort([f for f in os.listdir(DATASET_DIR) if f.endswith("650_bscan.png")])
files_1083 = numerical_sort([f for f in os.listdir(DATASET_DIR) if f.endswith("1083_bscan.png")])
files_1800 = numerical_sort([f for f in os.listdir(DATASET_DIR) if f.endswith("1800_bscan.png")])

psnr_stage1, ssim_stage1 = [], []
psnr_stage2, ssim_stage2 = [], []

pred_dir1 = os.path.join(DATASET_DIR, "predictions_stage1_huber+loss")
pred_dir2 = os.path.join(DATASET_DIR, "predictions_stage2_huber+loss")
os.makedirs(pred_dir1, exist_ok=True)
os.makedirs(pred_dir2, exist_ok=True)

for i in range(len(files_650)):
    # ---- Stage 1: 650 -> pred -> compare with 1083 ----
    lr_img = np.array(Image.open(os.path.join(DATASET_DIR, files_650[i])).resize(IMAGE_SIZE), dtype=np.float32) / 255.0
    gt_1083 = cv2.imread(os.path.join(DATASET_DIR, files_1083[i]), cv2.IMREAD_GRAYSCALE)

    pred_1083 = run_model(lr_img, model)
    Image.fromarray(pred_1083).save(os.path.join(pred_dir1, f"pred1_{i+1}.png"))

    if pred_1083.shape != gt_1083.shape:
        pred_1083 = cv2.resize(pred_1083, (gt_1083.shape[1], gt_1083.shape[0]))

    psnr_stage1.append(calculate_psnr(pred_1083, gt_1083))
    ssim_stage1.append(calculate_ssim(pred_1083, gt_1083))

    # ---- Stage 2: pred_1083 -> pred -> compare with 1800 ----
    gt_1800 = cv2.imread(os.path.join(DATASET_DIR, files_1800[i]), cv2.IMREAD_GRAYSCALE)

    # resize pred_1083 to IMAGE_SIZE before refeeding
    pred_1083_resized = cv2.resize(pred_1083, IMAGE_SIZE).astype(np.float32) / 255.0
    pred_1800 = run_model(pred_1083_resized, model)
    Image.fromarray(pred_1800).save(os.path.join(pred_dir2, f"pred2_{i+1}.png"))

    if pred_1800.shape != gt_1800.shape:
        pred_1800 = cv2.resize(pred_1800, (gt_1800.shape[1], gt_1800.shape[0]))

    psnr_stage2.append(calculate_psnr(pred_1800, gt_1800))
    ssim_stage2.append(calculate_ssim(pred_1800, gt_1800))

# =========================
# RESULTS
# =========================
print("\n---- Stage 1: 650 → pred → compare with 1083 ----")
print(f"SSIM: avg={np.mean(ssim_stage1):.4f}, min={np.min(ssim_stage1):.4f}, max={np.max(ssim_stage1):.4f}")
print(f"PSNR: avg={np.mean(psnr_stage1):.2f} dB, min={np.min(psnr_stage1):.2f} dB, max={np.max(psnr_stage1):.2f} dB")

print("\n---- Stage 2: pred_1083 → pred → compare with 1800 ----")
print(f"SSIM: avg={np.mean(ssim_stage2):.4f}, min={np.min(ssim_stage2):.4f}, max={np.max(ssim_stage2):.4f}")
print(f"PSNR: avg={np.mean(psnr_stage2):.2f} dB, min={np.min(psnr_stage2):.2f} dB, max={np.max(psnr_stage2):.2f} dB")



---- Stage 1: 650 → pred → compare with 1083 ----
SSIM: avg=0.9564, min=0.9340, max=0.9682
PSNR: avg=35.10 dB, min=33.55 dB, max=35.88 dB

---- Stage 2: pred_1083 → pred → compare with 1800 ----
SSIM: avg=0.9348, min=0.9068, max=0.9521
PSNR: avg=32.97 dB, min=30.48 dB, max=34.28 dB


In [5]:
import os
import re
import numpy as np
from PIL import Image
import cv2
from math import log10
from skimage.metrics import structural_similarity as ssim

import torch
import torch.nn as nn

# =========================
# CONFIG
# =========================
DATASET_DIR = r"C:\Preet\clean_validation dataset\png_images_900M_1500M_2500M"
MODEL_PATH = r"C:\Preet\clean_paired_bscans\best_attention_unet_final_huber+ssim.pth"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
IMAGE_SIZE = (256, 256)

# =========================
# MODEL BLOCKS
# =========================
class ConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.ReLU(inplace=True),
        )
    def forward(self, x): return self.conv(x)

class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(channels, channels, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),
            nn.Conv2d(channels, channels, 3, padding=1),
        )
    def forward(self, x): return x + self.conv(x)

class AttentionGate(nn.Module):
    def __init__(self, F_g, F_l, F_int):
        super().__init__()
        self.W_g = nn.Conv2d(F_g, F_int, 1)
        self.W_x = nn.Conv2d(F_l, F_int, 1)
        self.psi = nn.Sequential(
            nn.ReLU(inplace=True),
            nn.Conv2d(F_int, 1, 1),
            nn.Sigmoid()
        )
    def forward(self, x, g):
        g1 = self.W_g(g)
        x1 = self.W_x(x)
        psi = self.psi(g1 + x1)
        return x * psi

class UNet3WithAttention(nn.Module):
    def __init__(self):
        super().__init__()
        # Encoder
        self.enc1 = ConvBlock(1, 64); self.pool1 = nn.MaxPool2d(2)
        self.enc2 = ConvBlock(64, 128); self.pool2 = nn.MaxPool2d(2)
        self.enc3 = ConvBlock(128, 256); self.pool3 = nn.MaxPool2d(2)

        # Bottleneck
        self.bottleneck = nn.Sequential(*[ResidualBlock(256) for _ in range(6)])

        # Decoder
        self.up3 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.att3 = AttentionGate(256, 256, 128)
        self.dec3 = ConvBlock(512, 256)

        self.up2 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.att2 = AttentionGate(256, 128, 64)
        self.dec2 = ConvBlock(384, 128)

        self.up1 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.att1 = AttentionGate(128, 64, 32)
        self.dec1 = ConvBlock(192, 64)

        self.out_conv = nn.Conv2d(64, 1, 1)

    def forward(self, x):
        c1 = self.enc1(x); p1 = self.pool1(c1)
        c2 = self.enc2(p1); p2 = self.pool2(c2)
        c3 = self.enc3(p2); p3 = self.pool3(c3)
        b = self.bottleneck(p3)

        u3 = self.up3(b); a3 = self.att3(c3, u3); d3 = self.dec3(torch.cat([u3, a3], dim=1))
        u2 = self.up2(d3); a2 = self.att2(c2, u2); d2 = self.dec2(torch.cat([u2, a2], dim=1))
        u1 = self.up1(d2); a1 = self.att1(c1, u1); d1 = self.dec1(torch.cat([u1, a1], dim=1))
        return self.out_conv(d1)

# =========================
# HELPERS
# =========================
def calculate_psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0: return float('inf')
    return 20 * log10(255.0 / np.sqrt(mse))

def calculate_ssim(img1, img2):
    return ssim(img1, img2, data_range=255)

def numerical_sort(files):
    return sorted(files, key=lambda f: int(re.search(r'\d+', f).group()))

def run_model(img, model):
    tensor = torch.tensor(img, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(DEVICE)
    with torch.no_grad():
        pred = model(tensor).cpu().numpy()
    pred = np.squeeze(pred)
    pred = (pred * 255.0).clip(0, 255).astype(np.uint8)
    return cv2.normalize(pred, None, 0, 255, cv2.NORM_MINMAX)

# =========================
# LOAD MODEL
# =========================
model = UNet3WithAttention().to(DEVICE)
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.eval()

# =========================
# INFERENCE 2-STAGE
# =========================
files_650 = numerical_sort([f for f in os.listdir(DATASET_DIR) if f.endswith("900_bscan.png")])
files_1083 = numerical_sort([f for f in os.listdir(DATASET_DIR) if f.endswith("1500_bscan.png")])
files_1800 = numerical_sort([f for f in os.listdir(DATASET_DIR) if f.endswith("2500_bscan.png")])

psnr_stage1, ssim_stage1 = [], []
psnr_stage2, ssim_stage2 = [], []

pred_dir1 = os.path.join(DATASET_DIR, "predictions_stage1_huber+loss")
pred_dir2 = os.path.join(DATASET_DIR, "predictions_stage2_huber+loss")
os.makedirs(pred_dir1, exist_ok=True)
os.makedirs(pred_dir2, exist_ok=True)

for i in range(len(files_650)):
    # ---- Stage 1: 650 -> pred -> compare with 1083 ----
    lr_img = np.array(Image.open(os.path.join(DATASET_DIR, files_650[i])).resize(IMAGE_SIZE), dtype=np.float32) / 255.0
    gt_1083 = cv2.imread(os.path.join(DATASET_DIR, files_1083[i]), cv2.IMREAD_GRAYSCALE)

    pred_1083 = run_model(lr_img, model)
    Image.fromarray(pred_1083).save(os.path.join(pred_dir1, f"pred1_{i+1}.png"))

    if pred_1083.shape != gt_1083.shape:
        pred_1083 = cv2.resize(pred_1083, (gt_1083.shape[1], gt_1083.shape[0]))

    psnr_stage1.append(calculate_psnr(pred_1083, gt_1083))
    ssim_stage1.append(calculate_ssim(pred_1083, gt_1083))

    # ---- Stage 2: pred_1083 -> pred -> compare with 1800 ----
    gt_1800 = cv2.imread(os.path.join(DATASET_DIR, files_1800[i]), cv2.IMREAD_GRAYSCALE)

    # resize pred_1083 to IMAGE_SIZE before refeeding
    pred_1083_resized = cv2.resize(pred_1083, IMAGE_SIZE).astype(np.float32) / 255.0
    pred_1800 = run_model(pred_1083_resized, model)
    Image.fromarray(pred_1800).save(os.path.join(pred_dir2, f"pred2_{i+1}.png"))

    if pred_1800.shape != gt_1800.shape:
        pred_1800 = cv2.resize(pred_1800, (gt_1800.shape[1], gt_1800.shape[0]))

    psnr_stage2.append(calculate_psnr(pred_1800, gt_1800))
    ssim_stage2.append(calculate_ssim(pred_1800, gt_1800))

# =========================
# RESULTS
# =========================
print("\n---- Stage 1: 900 → pred → compare with 1500 ----")
print(f"SSIM: avg={np.mean(ssim_stage1):.4f}, min={np.min(ssim_stage1):.4f}, max={np.max(ssim_stage1):.4f}")
print(f"PSNR: avg={np.mean(psnr_stage1):.2f} dB, min={np.min(psnr_stage1):.2f} dB, max={np.max(psnr_stage1):.2f} dB")

print("\n---- Stage 2: pred_1500 → pred → compare with 2500 ----")
print(f"SSIM: avg={np.mean(ssim_stage2):.4f}, min={np.min(ssim_stage2):.4f}, max={np.max(ssim_stage2):.4f}")
print(f"PSNR: avg={np.mean(psnr_stage2):.2f} dB, min={np.min(psnr_stage2):.2f} dB, max={np.max(psnr_stage2):.2f} dB")



---- Stage 1: 900 → pred → compare with 1500 ----
SSIM: avg=0.7530, min=0.5993, max=0.8785
PSNR: avg=29.38 dB, min=28.78 dB, max=30.07 dB

---- Stage 2: pred_1500 → pred → compare with 2500 ----
SSIM: avg=0.7242, min=0.6028, max=0.8295
PSNR: avg=27.30 dB, min=26.92 dB, max=27.77 dB


testing 400Mhz on the trained on 750Mhz model.

In [1]:
import os
import re
import numpy as np
from PIL import Image
import cv2
from math import log10
from skimage.metrics import structural_similarity as ssim

import torch
import torch.nn as nn


# =========================
# CONFIG
# =========================
TEST_DIR = r"C:\Preet\400_670_Dataset"

MODEL_PATH = r"C:\Preet\clean_paired_bscans\best_attention_unet_final_huber+ssim.pth"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
IMAGE_SIZE = (256, 256)

# =========================
# MODEL BLOCKS (UNet3 + 6 RB)
# =========================
class ConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.ReLU(inplace=True),
        )
    def forward(self, x): return self.conv(x)

class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(channels, channels, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),
            nn.Conv2d(channels, channels, 3, padding=1),
        )
    def forward(self, x): return x + self.conv(x)

class AttentionGate(nn.Module):
    def __init__(self, F_g, F_l, F_int):
        super().__init__()
        self.W_g = nn.Conv2d(F_g, F_int, 1)
        self.W_x = nn.Conv2d(F_l, F_int, 1)
        self.psi = nn.Sequential(
            nn.ReLU(inplace=True),
            nn.Conv2d(F_int, 1, 1),
            nn.Sigmoid()
        )
    def forward(self, x, g):
        g1 = self.W_g(g)
        x1 = self.W_x(x)
        psi = self.psi(g1 + x1)
        return x * psi

class UNet3WithAttention(nn.Module):
    def __init__(self):
        super().__init__()
        # Encoder
        self.enc1 = ConvBlock(1, 64); self.pool1 = nn.MaxPool2d(2)
        self.enc2 = ConvBlock(64, 128); self.pool2 = nn.MaxPool2d(2)
        self.enc3 = ConvBlock(128, 256); self.pool3 = nn.MaxPool2d(2)

        # Bottleneck
        self.bottleneck = nn.Sequential(*[ResidualBlock(256) for _ in range(6)])

        # Decoder
        self.up3 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.att3 = AttentionGate(256, 256, 128)
        self.dec3 = ConvBlock(512, 256)

        self.up2 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.att2 = AttentionGate(256, 128, 64)
        self.dec2 = ConvBlock(384, 128)

        self.up1 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.att1 = AttentionGate(128, 64, 32)
        self.dec1 = ConvBlock(192, 64)

        self.out_conv = nn.Conv2d(64, 1, 1)

    def forward(self, x):
        c1 = self.enc1(x); p1 = self.pool1(c1)
        c2 = self.enc2(p1); p2 = self.pool2(c2)
        c3 = self.enc3(p2); p3 = self.pool3(c3)
        b = self.bottleneck(p3)

        u3 = self.up3(b); a3 = self.att3(c3, u3); d3 = self.dec3(torch.cat([u3, a3], dim=1))
        u2 = self.up2(d3); a2 = self.att2(c2, u2); d2 = self.dec2(torch.cat([u2, a2], dim=1))
        u1 = self.up1(d2); a1 = self.att1(c1, u1); d1 = self.dec1(torch.cat([u1, a1], dim=1))
        return self.out_conv(d1)

# =========================
# HELPERS
# =========================
def calculate_psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0: return float('inf')
    return 20 * log10(255.0 / np.sqrt(mse))

def calculate_ssim(img1, img2):
    return ssim(img1, img2, data_range=255)

def numerical_sort(files):
    return sorted(files, key=lambda f: int(re.search(r'\d+', f).group()))

# =========================
# LOAD MODEL
# =========================
model = UNet3WithAttention().to(DEVICE)
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.eval()


print(f"\n---- Evaluating {TEST_DIR} ----")
pred_dir = os.path.join(TEST_DIR, "predictions_unet+RB+AG+hybrid")
os.makedirs(pred_dir, exist_ok=True)

low_files = numerical_sort([f for f in os.listdir(TEST_DIR) if f.endswith("_l.png")])
high_files = numerical_sort([f for f in os.listdir(TEST_DIR) if f.endswith("_h.png")])

psnr_values, ssim_values = [], []

for i in range(len(low_files)):
    low_path = os.path.join(TEST_DIR, low_files[i])
    high_path = os.path.join(TEST_DIR, high_files[i])

    # Load LR
    lr_img = np.array(Image.open(low_path).resize(IMAGE_SIZE), dtype=np.float32) / 255.0
    lr_tensor = torch.tensor(lr_img).unsqueeze(0).unsqueeze(0).to(DEVICE)

    # Predict
    with torch.no_grad():
        pred = model(lr_tensor).cpu().squeeze().numpy()

    # Scale to 0–255
    pred_img = (np.clip(pred, 0.0, 1.0) * 255.0).astype(np.uint8)
    pred_path = os.path.join(pred_dir, f"pred_{i+1}.png")
    Image.fromarray(pred_img).save(pred_path)

    # Load GT
    gt_img = cv2.imread(high_path, cv2.IMREAD_GRAYSCALE)
    if pred_img.shape != gt_img.shape:
        pred_img = cv2.resize(pred_img, (gt_img.shape[1], gt_img.shape[0]))

    # Metrics
    psnr_values.append(calculate_psnr(pred_img, gt_img))
    ssim_values.append(calculate_ssim(pred_img, gt_img))

if psnr_values and ssim_values:
    print(f"SSIM: avg={np.mean(ssim_values):.4f}, min={np.min(ssim_values):.4f}, max={np.max(ssim_values):.4f}")
    print(f"PSNR: avg={np.mean(psnr_values):.2f} dB, min={np.min(psnr_values):.2f} dB, max={np.max(psnr_values):.2f} dB")
else:
    print("[Error] No valid *_l.png and *_h.png pairs found.")



---- Evaluating C:\Preet\400_670_Dataset ----
SSIM: avg=0.7513, min=0.6264, max=0.8171
PSNR: avg=30.22 dB, min=29.31 dB, max=30.96 dB
